In [1]:
import pandas as pd
import numpy as np
import os
import glob as gb
import sys
from decimal import Decimal
import datetime

anio=2023
#sys.path.insert(0, "/clases/bd/")
#from conexion import MyDatabase
#import conexion 
sys.path.insert(0, "../../")
from clases.bd.conexion2 import MyDatabase2
conn = MyDatabase2()

Conexion exitosa con la Base de datos:irvin_hisminsa


In [2]:
def consulta_covid(conn):
    adultos=conn.df(f"""SELECT nt.*, mp.id_tipo_documento,mp.numero_documento,mp.apellido_paterno_paciente,mp.apellido_materno_paciente,mp.nombres_paciente,mp.fecha_nacimiento,mp.genero,mp.id_etnia,mp.historia_clinica,mp.ficha_familiar,mp.ubigeo_nacimiento,mp.ubigeo_reniec,mp.domicilio_reniec,mp.ubigeo_declarado,mp.domicilio_declarado,mp.referencia_domicilio,mp.id_pais,mp.id_establecimiento,mp.fecha_alta,mp.fecha_modificacion FROM maestros.nominal_trama nt 
INNER  JOIN maestros.maestro_paciente mp ON mp.id_paciente =nt.id_paciente  
WHERE nt.anio={anio}
AND AGE(nt.fecha_atencion, mp.fecha_nacimiento) BETWEEN '30 years' AND '59 years';""")

    return adultos

In [3]:
#pd.options.display.max_columns = None
# ruta carpeta datos
#ruta_datos = os.path.abspath("./../data/")
#df_data = pd.read_csv(ruta_datos+'/Hisminsa_2022/consolidados/df_consolidado_22-02-2023_inner.csv', encoding='UTF-8')
df_data= consulta_covid(conn)
df_data.columns = df_data.columns.str.title()
df_data

e:\Irvin\Irvin\Python\Python-GERESA\modulos\adulto\../..\clases\bd\conexion2.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  s = pd.read_sql(sql, con=self.conn)


MemoryError: Unable to allocate 699. MiB for an array with shape (18, 5087116) and data type int64

In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4683629 entries, 0 to 4683628
Data columns (total 65 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   Id_Cita                       int64         
 1   Anio                          int64         
 2   Mes                           int64         
 3   Dia                           int64         
 4   Fecha_Atencion                object        
 5   Lote                          object        
 6   Num_Pag                       int64         
 7   Num_Reg                       int64         
 8   Id_Ups                        int64         
 9   Id_Establecimiento            int64         
 10  Id_Paciente                   object        
 11  Id_Personal                   int64         
 12  Id_Registrador                int64         
 13  Id_Financiador                object        
 14  Id_Condicion_Establecimiento  object        
 15  Id_Condicion_Servicio         ob

In [ ]:
df_data.isnull().sum()

Id_Cita                       0
Anio                          0
Mes                           0
Dia                           0
Fecha_Atencion                0
                         ...   
Referencia_Domicilio    4683629
Id_Pais                       0
Id_Establecimiento            0
Fecha_Alta              4174461
Fecha_Modificacion      4612628
Length: 65, dtype: int64

In [ ]:
# Funcion para calcular el total por genero y por grupo etareo de las actividades de etapa de vida adulto mayor
def curso_vida_adulto(df,**kwargs):
    # Parametros
    act = kwargs['actividad']
    dx_empiesa_con = kwargs['dx_empiesa_con']
    dx2_empiesa_con = kwargs['dx2_empiesa_con']
    dx = kwargs['dx']
    lab = kwargs['lab']
    lab2 = kwargs['lab2']
    dx2 = kwargs['dx2']
    dx3 = kwargs['dx3']
    dx4 = kwargs['dx4']
    td = kwargs['td']
    td2 = kwargs['td2']
    td3 = kwargs['td3']
    enclab = kwargs['enclab']
    enclab2 = kwargs['enclab2']
    enclab_menos_en = kwargs['enclab_menos_en']
    cualquier_dx2 = kwargs['cualquier_dx2']
    gestantes = kwargs['gestantes']
    lab_isnull = kwargs['lab_isnull']
    #grupos etareos adultero mayor
    #list_ge = [[30,59],[30,59]]
    # lista de dataframes resultados segun grupo etareo
    #list=[]
    # Filtroltos por edad etarea, dx, y labs
    #for i in list_ge:
    df_GE = df[df['Anio_Actual_Paciente'].between(30,59)]
    if td:
        if lab:
            df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx)) & ((df_GE.Valor_Lab.isin(lab)))]
            if gestantes:
                if enclab:
                    df_geB = df_GE[(df_GE.Genero=='F') | (df_GE.Valor_Lab.isin(enclab))]
                    df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
                    df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente']) 
                    
                if lab2:
                    df_geB = df_GE[(df_GE.Valor_Lab.isin(lab2))]
                    df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
                    #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                # if enclab_menos_en:
                #     df_geB = df_GE[(df_GE.Genero=='F') | (~(df_GE.Valor_Lab.isin(enclab_menos_en)))]
                #     df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
                #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
            else: # no gestantes
                if enclab_menos_en:
                    # df_x = df_GE[(df_GE.Genero=='F') | (df_GE.Valor_Lab.isin(enclab_menos_en))].index
                    # df_y = pd.merge(df_PAMS,df_x, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
                    # df_xy = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                   
                    df_geB = df_GE[(~(df_GE.Valor_Lab.isin(enclab_menos_en)))]
                    df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
                    df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                    
                    # df_PAMS = pd.merge(df_PAMS,df_xy, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
                    # df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                    
                    # df_PAMS = df_PAMS.drop(df_xy.index)
                    
                    #df_PAMS = df_PAMS[df_PAMS.Id_Ups_x==303304]
                    #df_PAMS = df_PAMS[df_PAMS.Id_Profesion_x.isin([25,47])]  
                if enclab:
                    df_geB = df_GE[(df_GE.Valor_Lab.isin(enclab))]
                    df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
                    #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                if lab2:
                    df_geB = df_GE[(df_GE.Valor_Lab.isin(lab2))]
                    df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
                    #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
        else:
            if enclab:
                df_geA = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx))]
                if enclab_menos_en:
                    df_geB = df_GE[~(df_GE.Valor_Lab.isin(enclab_menos_en))] #ojo se modifico enclab
                    df_PAMS = pd.merge(df_geA,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Codigo_Item','Valor_Lab','Id_Correlativo','Id_Correlativo_Lab'])
                    df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                else:
                    df_geB = df_GE[(df_GE.Valor_Lab.isin(enclab))]
                    df_PAMS = pd.merge(df_geA,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'])
                    df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])  
            elif dx_empiesa_con:
                list_tmp=[]
                for k in dx_empiesa_con:
                    df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.str.startswith(k, na=False))]
                    list_tmp.append(df_PAMS)
                if len(list_tmp)==0:
                    #df_PAMS = pd.DataFrame(columns=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Nro'], index=range(1))
                    df_PAMS.empty = True
                elif len(list_tmp)==1:
                    df_PAMS = list_tmp[0]
                elif len(list_tmp)==2:
                    df_PAMS = pd.merge(list_tmp[0],list_tmp[1], on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='outer')
                elif len(list_tmp)==3:
                    df_PAMS = list_tmp[0].merge(list_tmp[1], on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner').merge(list_tmp[2], on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='outer')
                    #df_PAMS = pd.merge(df_PAMS,list_tmp[j+1], on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'])
                    # df_PAMS = pd.concat(list_tmp, ignore_index=True)
                #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])   
            else:
                #df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.str.startswith(dx[0], na=False))]
                df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx))]
                # if cualquier_cod:
                #     w = df_GE[(df_GE.Codigo_Item.isin(dx))]
                #     y = pd.merge(df_PAMS,w, on=['Id_Cita','Anio','Mes','Genero'], how='left')
                #     d1=df_GE[(df_GE.Codigo_Item.isin(dx))]
                #     d2=df_GE[(df_GE.Codigo_Item.isin(['99401']))]
                #     d = pd.merge(d1,d2, on=['Id_Cita','Anio','Mes','Genero'])
                #     df_gem = pd.merge(y,d, on=['Id_Cita','Anio','Mes','Genero'], how='left')
                #     df_PAMS = df_gem.drop_duplicates(subset=['Id_Cita'])
    elif lab:
        df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx)) & (df_GE.Valor_Lab.isin(lab))]
    else:
        if lab_isnull:
            df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx)) & (df_GE.Valor_Lab.isnull())]   
        else:
            df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx))]
        # df_bl = df_GE[(df_GE.Valor_Lab.isnull())]
        # df_PAMS = pd.merge(df_PAMS1,df_bl, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
        

    # if dx2:
    #     df_PAMS1 = df_GE[df_GE.Codigo_Item.isin(dx2)]
    #     df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Anio','Mes','Genero'], how='inner')
        
    if td2 and dx2 and lab2:
        df_PAMS2 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.isin(dx2)) & ((df_GE.Valor_Lab.isin(lab2)))]
        df_PAMS = pd.merge(df_PAMS,df_PAMS2, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
        df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
    elif td2 and dx2:
        df_PAMS1 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.isin(dx2))]   
        df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Id_Paciente','Anio','Mes','Genero'], how='inner')
    elif td2:
        if ((cualquier_dx2) and (dx2_empiesa_con)):
            df_vcfr1 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.isin(cualquier_dx2))]
            df_vcfr2 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.str.startswith(dx2_empiesa_con[0], na=False))]
            df_PAMS1 = pd.merge(df_vcfr1, df_vcfr2, on=["Id_Cita",'Id_Paciente','Anio','Mes','Genero'], how='outer')
            df_PAMS1 = df_PAMS1.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
            # else:
            #     df_PAMS1 = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(cualquier_dx))]
            #df_PAMS2 = df_GE[(df_GE.Codigo_Item.str.startswith(dx_empiesa_con[0], na=False))]
        # elif dx2:
        #     df_PAMS1 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.isin(dx2))]   
            df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Id_Paciente','Anio','Mes','Genero'], how='inner')
    elif dx2:
        if enclab2:
            df_PAMS1 = df_GE[(df_GE.Codigo_Item.isin(dx2)) & (df_GE.Valor_Lab.isin(enclab2))]   
            df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Id_Paciente','Anio','Mes','Genero'], how='inner')          
        else:    
            df_PAMS1 = df_GE[(df_GE.Codigo_Item.isin(dx2))]   
            df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Id_Paciente','Anio','Mes','Genero'], how='inner')
    else: #dx2 verdad
    # if lab2:
    #     df_geB = df_GE[(df_GE.Valor_Lab.isin(lab2))]
    #     df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
    #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])

    # if gestante:
    #     df_geB = df_GE[(df_GE.Genero=='F') & (df_GE.Valor_Lab=='G')]
    #     df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
    #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
    #     if lab2:
    #         df_z = df_GE[(df_GE.Valor_Lab.isin(lab2))]
    #         df_PAMS = pd.merge(df_PAMS,df_z, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
    #         df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])

    # if enclab_menos_en:
    #     df_geB = df_GE[~(df_GE.Valor_Lab.isin(enclab_menos_en))]
    #     df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Codigo_Item','Valor_Lab','Id_Correlativo','Id_Correlativo_Lab'])
    #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Codigo_Item','Valor_Lab','Id_Correlativo','Id_Correlativo_Lab'])
    
        if enclab2:
            df_geB = df_GE[(df_GE.Valor_Lab.isin(enclab2))]
            df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
            df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente']) 
    
    if td3 and dx3:
        df_PAMS3 = df_GE[(df_GE.Tipo_Diagnostico.isin(td3)) & (df_GE.Codigo_Item.isin(dx3))]  
        df_PAMS = pd.merge(df_PAMS,df_PAMS3, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
        df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
        
    if dx4:
        df_PAMS4 = df_GE[(df_GE.Codigo_Item.isin(dx4))]  
        df_PAMS = pd.merge(df_PAMS,df_PAMS4, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
        df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
        
    
    if not (df_PAMS.empty):
        # agrupamos y contamos por genero
        df_PAMS = df_PAMS.groupby(['Anio','Mes','Genero']).size().reset_index(name='Nro')
        # transponemos genero
        df_PAMS = df_PAMS.pivot_table(index=['Anio','Mes'], columns='Genero', values='Nro', aggfunc=np.sum, fill_value=0).reset_index()
        if not (df_PAMS.empty):
            if set(['M','F']).issubset(df_PAMS.columns):
                # suma de los generos por meses
                df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(
                    Masculino = ('M', sum),
                    Femenino = ('F', sum)
                ).reset_index()
            elif 'M' in df_PAMS.columns:
                # suma solo masculinos
                df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(
                    Masculino = ('M', sum),
                ).reset_index()
                df_PAMS = df_PAMS.assign(Femenino=0)
                #print(df_PAMS)
            else:
                # suma solo femeninos
                df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(
                    Femenino = ('F', sum)
                ).reset_index()
                df_PAMS = df_PAMS.assign(Masculino=0)
                df_PAMS = df_PAMS[['Anio','Mes','Masculino','Femenino']]
            #list.append(df_PAMS)
        else:
            df_PAMS = pd.DataFrame(columns=['Anio','Mes','Masculino','Femenino'], index=range(1))
        #list.append(df_PAMS)
    else:
        df_PAMS = pd.DataFrame(columns=['Anio','Mes','Masculino','Femenino'], index=range(1))
    #list.append(df_PAMS)   

    #df_PAMS = pd.merge(list[0], list[1], on=["Anio","Mes"],how='outer',suffixes=("_30_59","_30_59")).fillna(0)
    #df_PAMS = pd.merge(df_PAMS, list[2], on=["Anio","Mes"],how='outer').fillna(0)
    # if set(['Masculino_30_59','Masculino_30_59','Femenino_30_59','Femenino_30_59']).issubset(df_PAMS.columns):
    #     df_PAMS['Total_Masculinos'] = df_PAMS['Masculino_60_69']+df_PAMS['Masculino_70_79']+df_PAMS['Masculino']
    #     df_PAMS['Total_Femeninos'] = df_PAMS['Femenino_60_69']+df_PAMS['Femenino_70_79']+df_PAMS['Femenino']
    # elif set(['Masculino_60_69','Masculino_70_79','Masculino']).issubset(df_PAMS.columns):
    #     df_PAMS['Total_Masculinos'] = df_PAMS['Masculino_60_69']+df_PAMS['Masculino_70_79']+df_PAMS['Masculino']
    # else:
    df_PAMS['Total'] = df_PAMS['Masculino']+df_PAMS['Femenino']
    df_PAMS = df_PAMS[~df_PAMS.Mes.isnull()]
    df_PAMS = df_PAMS.astype(int)
    df_PAMS =  df_PAMS.sort_values(['Anio','Mes'])
    df_PAMS = df_PAMS.assign(Actividad=act)
    df_PAMS = df_PAMS[df_PAMS.Mes!=0]

    return df_PAMS

In [ ]:
# Funcion para calcular el total por genero y por grupo etareo de las actividades de etapa de vida adulto mayor
def curso_vida_adulto_general(df,**kwargs):
    # Parametros
    act = kwargs['actividad']
    dx_empiesa_con = kwargs['dx_empiesa_con']
    dx2_empiesa_con = kwargs['dx2_empiesa_con']
    dx = kwargs['dx']
    lab = kwargs['lab']
    lab2 = kwargs['lab2']
    dx2 = kwargs['dx2']
    dx3 = kwargs['dx3']
    dx4 = kwargs['dx4']
    td = kwargs['td']
    td2 = kwargs['td2']
    td3 = kwargs['td3']
    enclab = kwargs['enclab']
    enclab2 = kwargs['enclab2']
    enclab_menos_en = kwargs['enclab_menos_en']
    cualquier_dx2 = kwargs['cualquier_dx2']
    gestantes = kwargs['gestantes']
    lab_isnull = kwargs['lab_isnull']
    #grupos etareos adultero mayor
    #list_ge = [[30,59],[30,59]]
    # lista de dataframes resultados segun grupo etareo
    #list=[]
    # Filtroltos por edad etarea, dx, y labs
    #for i in list_ge:
    df_GE = df[df['Anio_Actual_Paciente'].between(30,59)]
    if td:
        if lab:
            df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx)) & ((df_GE.Valor_Lab.isin(lab)))]
            if gestantes:
                if enclab:
                    df_geB = df_GE[(df_GE.Genero=='F') | (df_GE.Valor_Lab.isin(enclab))]
                    df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'],how='inner')
                    df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente']) 
                    
                if lab2:
                    df_geB = df_GE[(df_GE.Valor_Lab.isin(lab2))]
                    df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'],how='inner')
                    #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                # if enclab_menos_en:
                #     df_geB = df_GE[(df_GE.Genero=='F') | (~(df_GE.Valor_Lab.isin(enclab_menos_en)))]
                #     df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
                #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
            else: # no gestantes
                if enclab_menos_en:
                    # df_x = df_GE[(df_GE.Genero=='F') | (df_GE.Valor_Lab.isin(enclab_menos_en))].index
                    # df_y = pd.merge(df_PAMS,df_x, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
                    # df_xy = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                   
                    df_geB = df_GE[(~(df_GE.Valor_Lab.isin(enclab_menos_en)))]
                    df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='inner')
                    df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                    
                    # df_PAMS = pd.merge(df_PAMS,df_xy, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
                    # df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                    
                    # df_PAMS = df_PAMS.drop(df_xy.index)
                    
                    #df_PAMS = df_PAMS[df_PAMS.Id_Ups_x==303304]
                    #df_PAMS = df_PAMS[df_PAMS.Id_Profesion_x.isin([25,47])]  
                if enclab:
                    df_geB = df_GE[(df_GE.Valor_Lab.isin(enclab))]
                    df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'],how='inner')
                    #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                if lab2:
                    df_geB = df_GE[(df_GE.Valor_Lab.isin(lab2))]
                    df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'],how='inner')
                    #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
        else:
            if enclab:
                df_geA = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx))]
                if enclab_menos_en:
                    df_geB = df_GE[~(df_GE.Valor_Lab.isin(enclab_menos_en))] #ojo se modifico enclab
                    df_PAMS = pd.merge(df_geA,df_geB, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero','Codigo_Item','Valor_Lab','Id_Correlativo','Id_Correlativo_Lab'])
                    df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                else:
                    df_geB = df_GE[(df_GE.Valor_Lab.isin(enclab))]
                    df_PAMS = pd.merge(df_geA,df_geB, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'])
                    df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])  
            elif dx_empiesa_con:
                list_tmp=[]
                for k in dx_empiesa_con:
                    df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.str.startswith(k, na=False))]
                    list_tmp.append(df_PAMS)
                if len(list_tmp)==0:
                    #df_PAMS = pd.DataFrame(columns=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Nro'], index=range(1))
                    df_PAMS.empty = True
                elif len(list_tmp)==1:
                    df_PAMS = list_tmp[0]
                elif len(list_tmp)==2:
                    df_PAMS = pd.merge(list_tmp[0],list_tmp[1], on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='outer')
                elif len(list_tmp)==3:
                    df_PAMS = list_tmp[0].merge(list_tmp[1], on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='inner').merge(list_tmp[2], on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='outer')
                    #df_PAMS = pd.merge(df_PAMS,list_tmp[j+1], on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'])
                    # df_PAMS = pd.concat(list_tmp, ignore_index=True)
                #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])   
            else:
                #df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.str.startswith(dx[0], na=False))]
                df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx))]
                # if cualquier_cod:
                #     w = df_GE[(df_GE.Codigo_Item.isin(dx))]
                #     y = pd.merge(df_PAMS,w, on=['Id_Cita','Anio','Mes','Genero'], how='left')
                #     d1=df_GE[(df_GE.Codigo_Item.isin(dx))]
                #     d2=df_GE[(df_GE.Codigo_Item.isin(['99401']))]
                #     d = pd.merge(d1,d2, on=['Id_Cita','Anio','Mes','Genero'])
                #     df_gem = pd.merge(y,d, on=['Id_Cita','Anio','Mes','Genero'], how='left')
                #     df_PAMS = df_gem.drop_duplicates(subset=['Id_Cita'])
    elif lab:
        df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx)) & (df_GE.Valor_Lab.isin(lab))]
    else:
        if lab_isnull:
            df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx)) & (df_GE.Valor_Lab.isnull())]   
        else:
            df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx))]
        # df_bl = df_GE[(df_GE.Valor_Lab.isnull())]
        # df_PAMS = pd.merge(df_PAMS1,df_bl, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
        

    # if dx2:
    #     df_PAMS1 = df_GE[df_GE.Codigo_Item.isin(dx2)]
    #     df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Anio','Mes','Genero'], how='inner')
        
    if td2 and dx2 and lab2:
        df_PAMS2 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.isin(dx2)) & ((df_GE.Valor_Lab.isin(lab2)))]
        df_PAMS = pd.merge(df_PAMS,df_PAMS2, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'],how='inner')
        df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
    elif td2 and dx2:
        df_PAMS1 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.isin(dx2))]   
        df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='inner')
    elif td2:
        if ((cualquier_dx2) and (dx2_empiesa_con)):
            df_vcfr1 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.isin(cualquier_dx2))]
            df_vcfr2 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.str.startswith(dx2_empiesa_con[0], na=False))]
            df_PAMS1 = pd.merge(df_vcfr1, df_vcfr2, on=["Id_Cita",'Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='outer')
            df_PAMS1 = df_PAMS1.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
            # else:
            #     df_PAMS1 = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(cualquier_dx))]
            #df_PAMS2 = df_GE[(df_GE.Codigo_Item.str.startswith(dx_empiesa_con[0], na=False))]
        # elif dx2:
        #     df_PAMS1 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.isin(dx2))]   
            df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='inner')
    elif dx2:
        if enclab2:
            df_PAMS1 = df_GE[(df_GE.Codigo_Item.isin(dx2)) & (df_GE.Valor_Lab.isin(enclab2))]   
            df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='inner')          
        else:    
            df_PAMS1 = df_GE[(df_GE.Codigo_Item.isin(dx2))]   
            df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='inner')
    else: #dx2 verdad
    # if lab2:
    #     df_geB = df_GE[(df_GE.Valor_Lab.isin(lab2))]
    #     df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
    #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])

    # if gestante:
    #     df_geB = df_GE[(df_GE.Genero=='F') & (df_GE.Valor_Lab=='G')]
    #     df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
    #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
    #     if lab2:
    #         df_z = df_GE[(df_GE.Valor_Lab.isin(lab2))]
    #         df_PAMS = pd.merge(df_PAMS,df_z, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
    #         df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])

    # if enclab_menos_en:
    #     df_geB = df_GE[~(df_GE.Valor_Lab.isin(enclab_menos_en))]
    #     df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Codigo_Item','Valor_Lab','Id_Correlativo','Id_Correlativo_Lab'])
    #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Codigo_Item','Valor_Lab','Id_Correlativo','Id_Correlativo_Lab'])
    
        if enclab2:
            df_geB = df_GE[(df_GE.Valor_Lab.isin(enclab2))]
            df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'],how='inner')
            df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente']) 
    
    if td3 and dx3:
        df_PAMS3 = df_GE[(df_GE.Tipo_Diagnostico.isin(td3)) & (df_GE.Codigo_Item.isin(dx3))]  
        df_PAMS = pd.merge(df_PAMS,df_PAMS3, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'],how='inner')
        df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
        
    if dx4:
        df_PAMS4 = df_GE[(df_GE.Codigo_Item.isin(dx4))]  
        df_PAMS = pd.merge(df_PAMS,df_PAMS4, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'],how='inner')
        df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
        
    
    if not (df_PAMS.empty):
        # agrupamos y contamos por genero
        df_PAMS = df_PAMS.groupby(['Id_Paciente','Id_Establecimiento','Anio','Mes','Genero']).size().reset_index(name='Nro')
        # transponemos genero
        df_PAMS = df_PAMS.pivot_table(index=['Id_Paciente','Id_Establecimiento','Anio','Mes'], columns='Genero', values='Nro', aggfunc=np.sum, fill_value=0).reset_index()
        if not (df_PAMS.empty):
            if set(['M','F']).issubset(df_PAMS.columns):
                # suma de los generos por meses
                df_PAMS=df_PAMS.groupby(['Id_Paciente','Id_Establecimiento','Anio','Mes']).agg(
                    Masculino = ('M', sum),
                    Femenino = ('F', sum)
                ).reset_index()
            elif 'M' in df_PAMS.columns:
                # suma solo masculinos
                df_PAMS=df_PAMS.groupby(['Id_Paciente','Id_Establecimiento','Anio','Mes']).agg(
                    Masculino = ('M', sum),
                ).reset_index()
                df_PAMS = df_PAMS.assign(Femenino=0)
                #print(df_PAMS)
            else:
                # suma solo femeninos
                df_PAMS=df_PAMS.groupby(['Id_Paciente','Id_Establecimiento','Anio','Mes']).agg(
                    Femenino = ('F', sum)
                ).reset_index()
                df_PAMS = df_PAMS.assign(Masculino=0)
                df_PAMS = df_PAMS[['Id_Paciente','Id_Establecimiento','Anio','Mes','Masculino','Femenino']]
            #list.append(df_PAMS)
        else:
            df_PAMS = pd.DataFrame(columns=['Id_Paciente','Id_Establecimiento','Anio','Mes','Masculino','Femenino'], index=range(1))
        #list.append(df_PAMS)
    else:
        df_PAMS = pd.DataFrame(columns=['Id_Paciente','Id_Establecimiento','Anio','Mes','Masculino','Femenino'], index=range(1))
    #list.append(df_PAMS)   

    #df_PAMS = pd.merge(list[0], list[1], on=["Anio","Mes"],how='outer',suffixes=("_30_59","_30_59")).fillna(0)
    #df_PAMS = pd.merge(df_PAMS, list[2], on=["Anio","Mes"],how='outer').fillna(0)
    # if set(['Masculino_30_59','Masculino_30_59','Femenino_30_59','Femenino_30_59']).issubset(df_PAMS.columns):
    #     df_PAMS['Total_Masculinos'] = df_PAMS['Masculino_60_69']+df_PAMS['Masculino_70_79']+df_PAMS['Masculino']
    #     df_PAMS['Total_Femeninos'] = df_PAMS['Femenino_60_69']+df_PAMS['Femenino_70_79']+df_PAMS['Femenino']
    # elif set(['Masculino_60_69','Masculino_70_79','Masculino']).issubset(df_PAMS.columns):
    #     df_PAMS['Total_Masculinos'] = df_PAMS['Masculino_60_69']+df_PAMS['Masculino_70_79']+df_PAMS['Masculino']
    # else:
    df_PAMS['Total'] = df_PAMS['Masculino']+df_PAMS['Femenino']
    df_PAMS = df_PAMS[~df_PAMS.Mes.isnull()]
    df_PAMS = df_PAMS.astype(int)
    df_PAMS =  df_PAMS.sort_values(['Anio','Mes'])
    df_PAMS = df_PAMS.assign(Actividad=act)
    df_PAMS = df_PAMS[df_PAMS.Mes!=0]

    return df_PAMS

In [ ]:
etsne = curso_vida_adulto(df_data, actividad ='001.Enf. de trans. Sex. no especificada', td=['D'], dx_empiesa_con=['A64'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
etsne

C:\Users\IRVIN\AppData\Local\Temp\ipykernel_16128\2930417130.py:191: FutureWarning: The provided callable <function sum at 0x000001FB36208900> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_PAMS = df_PAMS.pivot_table(index=['Anio','Mes'], columns='Genero', values='Nro', aggfunc=np.sum, fill_value=0).reset_index()
C:\Users\IRVIN\AppData\Local\Temp\ipykernel_16128\2930417130.py:195: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(


,Anio,Mes,Masculino,Femenino,Total,Actividad
0,2023,1,9,1152,1161,001.Enf. de trans. Sex. no especificada
1,2023,2,11,1583,1594,001.Enf. de trans. Sex. no especificada
2,2023,3,34,1555,1589,001.Enf. de trans. Sex. no especificada
3,2023,4,7,1195,1202,001.Enf. de trans. Sex. no especificada
4,2023,5,5,1158,1163,001.Enf. de trans. Sex. no especificada
5,2023,6,13,1092,1105,001.Enf. de trans. Sex. no especificada
6,2023,7,7,1219,1226,001.Enf. de trans. Sex. no especificada
7,2023,8,10,1175,1185,001.Enf. de trans. Sex. no especificada
8,2023,9,8,1303,1311,001.Enf. de trans. Sex. no especificada
9,2023,10,19,1232,1251,001.Enf. de trans. Sex. no especificada


In [ ]:
obesidad = curso_vida_adulto(df_data, actividad ='002.Obesidad', td=['D'], dx_empiesa_con=['E66'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
obesidad

C:\Users\IRVIN\AppData\Local\Temp\ipykernel_16128\2930417130.py:191: FutureWarning: The provided callable <function sum at 0x000001FB36208900> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_PAMS = df_PAMS.pivot_table(index=['Anio','Mes'], columns='Genero', values='Nro', aggfunc=np.sum, fill_value=0).reset_index()
C:\Users\IRVIN\AppData\Local\Temp\ipykernel_16128\2930417130.py:195: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(


,Anio,Mes,Masculino,Femenino,Total,Actividad
0,2023,1,534,2194,2728,002.Obesidad
1,2023,2,590,2545,3135,002.Obesidad
2,2023,3,659,2805,3464,002.Obesidad
3,2023,4,533,2059,2592,002.Obesidad
4,2023,5,733,2261,2994,002.Obesidad
5,2023,6,443,1647,2090,002.Obesidad
6,2023,7,468,1784,2252,002.Obesidad
7,2023,8,506,1889,2395,002.Obesidad
8,2023,9,403,1590,1993,002.Obesidad
9,2023,10,374,1675,2049,002.Obesidad


In [ ]:
# MORBILIDAD DEL ADULTO consolidado
etsne = curso_vida_adulto(df_data, actividad ='001.Enf. de trans. Sex. no especificada', td=['D'], dx_empiesa_con=['A64'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
obesidad = curso_vida_adulto(df_data, actividad ='002.Obesidad', td=['D'], dx_empiesa_con=['E66'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
rinofaringitis_aguda = curso_vida_adulto(df_data, actividad ='003.rinofaringitis aguda', td=['D'], dx_empiesa_con=['J00'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
faringitis_aguda = curso_vida_adulto(df_data, actividad ='004.Faringitis aguda', td=['D'], dx_empiesa_con=['J02'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
infecciones_agudas = curso_vida_adulto(df_data, actividad ='005.Infecciones agudas vias respiratorias superiores', td=['D'], dx_empiesa_con=['J06'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
caries_dental = curso_vida_adulto(df_data, actividad ='006.Caries dental', td=['D'], dx_empiesa_con=['K02'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
enfermedades_pulpa = curso_vida_adulto(df_data, actividad ='007.Enfer. de la pulpa y de los tejidos periapicales', td=['D'], dx_empiesa_con=['K04'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
gastritis_duodentis = curso_vida_adulto(df_data, actividad ='008.Gastritis y duodenitis', td=['D'], dx_empiesa_con=['K29'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dorsalgia = curso_vida_adulto(df_data, actividad ='009.Dorsalgia', td=['D'], dx_empiesa_con=['M54'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
Otros_transt_sist_uri = curso_vida_adulto(df_data, actividad ='010.Otros transtornos del sistema urinario', td=['D'], dx_empiesa_con=['N39'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# PLAN DE ATENCION INTEGRAL DEL ADULTO
plan_atencion_elaborado = curso_vida_adulto(df_data, actividad ='011.Plan de Atención Elaborado', td=[], dx_empiesa_con=[], dx=['C8002'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
plan_atencion_ejecutado = curso_vida_adulto(df_data, actividad ='012.Plan de Atención Ejecutado', td=[], dx_empiesa_con=[], dx=['C8002'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# EVALUACIÓN ANTROPOMÉTRICA
imc_delgadezIII = curso_vida_adulto(df_data, actividad ='013.Evaluación antropométrica IMC Delgadez III', td=['D'], dx_empiesa_con=[], dx=['E46X3'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_delgadezII = curso_vida_adulto(df_data, actividad ='014.Evaluación antropométrica IMC Delgadez II', td=['D'], dx_empiesa_con=[], dx=['E46X2'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_delgadezI = curso_vida_adulto(df_data, actividad ='015.Evaluación antropométrica IMC Delgadez I', td=['D'], dx_empiesa_con=[], dx=['E46X1'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_normal = curso_vida_adulto(df_data, actividad ='016.Evaluación antropométrica IMC Normal', td=['D'], dx_empiesa_con=[], dx=['Z006'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_sobrepeso = curso_vida_adulto(df_data, actividad ='017.Evaluación antropométrica IMC Sobrepeso', td=['D'], dx_empiesa_con=[], dx=['E660'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_obesidadI = curso_vida_adulto(df_data, actividad ='018.Evaluación antropométrica IMC Obesidad I', td=['D'], dx_empiesa_con=[], dx=['E669'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_obesidadII = curso_vida_adulto(df_data, actividad ='019.Evaluación antropométrica IMC Obesidad II', td=['D'], dx_empiesa_con=[], dx=['E6692'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_obesidadIII = curso_vida_adulto(df_data, actividad ='020.Evaluación antropométrica IMC Obesidad III', td=['D'], dx_empiesa_con=[], dx=['E6693'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# RIESGO NUTRICIONAL
rn_pab_normal = curso_vida_adulto(df_data, actividad ='021.Riesgo nutricional PAB Normal', td=['D'], dx_empiesa_con=[], dx=['U8170'], cualquier_dx=[], lab=['N'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
rn_pab_anormal = curso_vida_adulto(df_data, actividad ='022.Riesgo nutricional PAB Anormal', td=['D'], dx_empiesa_con=[], dx=['U8170'], cualquier_dx=[], lab=['A'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# <h1> DAÑOS NO TRANSMISIBLE </h1>
dnt_val_clinica_sin_fr = curso_vida_adulto(df_data, actividad ='023.Daños No Transmisibles Valoración Clínica Sin Factores de Riesgo', td=['D'], dx_empiesa_con=[], dx=['Z019'], cualquier_dx=[], lab=['DNT'], enclab=[],  td2=['D'], dx2_empiesa_con=[], dx2=['Z006'], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dnt_val_clinica_con_fr = curso_vida_adulto(df_data, actividad ='024.Daños No Transmisibles Valoración Clínica con Factores de Riesgo', td=['D'], dx_empiesa_con=[], dx=['Z019'], cualquier_dx=[], lab=['DNT'], enclab=[],  td2=['D'], dx2_empiesa_con=['E66'], dx2=[], cualquier_dx2=['E65X','Z720','Z721','Z723','Z724','Z883','Z834'], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dnt_val_clinica_tamizajeLab = curso_vida_adulto(df_data, actividad ='025.Daños No Transmisibles Valoración Clínica y tamizaje de lab.', td=['D'], dx_empiesa_con=[], dx=['U262'], cualquier_dx=[], lab=['DNT'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# casos diagnosticados HTA
dxHTA_hipertension_esencial = curso_vida_adulto(df_data, actividad ='026.DX HTA Hipertensión Esencial (primaria)', td=['D'], dx_empiesa_con=[], dx=['I10X'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_retinopatia_hipertensiva = curso_vida_adulto(df_data, actividad ='027.DX HTA Retinopatía Hipertensiva', td=['D'], dx_empiesa_con=[], dx=['H350'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_nefropatia_hipertensiva = curso_vida_adulto(df_data, actividad ='028.DX HTA Nefropatía Hipertensiva', td=['D'], dx_empiesa_con=[], dx=['I120','I129'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# Casos diagnosticados diabetes mellitus
dxHTA_diabetes_mellitus_tipo1 = curso_vida_adulto(df_data, actividad ='029.DX Diabetes Mellitus Tipo 1', td=['D'], dx_empiesa_con=['E10'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_diabetes_mellitus_tipo2 = curso_vida_adulto(df_data, actividad ='030.DX Diabetes Mellitus Tipo 2', td=['D'], dx_empiesa_con=['E11'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_diabetes_mellitus_embarazo = curso_vida_adulto(df_data, actividad ='031.DX Diabetes Mellitus en el Embarazo', td=['D'], dx_empiesa_con=['O24'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_otras_diabetes_mellitus = curso_vida_adulto(df_data, actividad ='032.DX Otras Diabetes Mellitus', td=['D'], dx_empiesa_con=['E12','E13','E14'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_nefro_diabetica = curso_vida_adulto(df_data, actividad ='033.DX Nefropatia Diabética', td=['D'], dx_empiesa_con=[], dx=['E102', 'E112', 'E122', 'E132', 'E142'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_retinop_diabetica = curso_vida_adulto(df_data, actividad ='034.DX Retinopatia Diabética', td=['D'], dx_empiesa_con=[], dx=['E103', 'E113', 'E123', 'E133', 'E143'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# <h1>PLANIFICACION FAMILIAR</h1>
consejeria_plan_family1 = curso_vida_adulto(df_data, actividad ='035.Consejería en planificación familiar 1ra sesion', td=['D'], dx_empiesa_con=[], dx=['99402.04'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_plan_family2 = curso_vida_adulto(df_data, actividad ='036.Consejería en planificación familiar 1ra sesion', td=['D'], dx_empiesa_con=[], dx=['99402.04'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_salud_sexual1 = curso_vida_adulto(df_data, actividad ='037.Consejería en salud sexual y reproductiva 1ra sesion', td=['D'], dx_empiesa_con=[], dx=['99402.03'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_salud_sexual2 = curso_vida_adulto(df_data, actividad ='038.Consejería en salud sexual y reproductiva 2da sesion', td=['D'], dx_empiesa_con=[], dx=['99402.03'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_perimeno_menopa_climat1 = curso_vida_adulto(df_data, actividad ='039.Consejería en perimenopausia, menopausia y climaterio 1ra sesion', td=['D'], dx_empiesa_con=[], dx=['99402.06'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_perimeno_menopa_climat2 = curso_vida_adulto(df_data, actividad ='040.Consejería en perimenopausia, menopausia y climaterio 2da sesion', td=['D'], dx_empiesa_con=[], dx=['99402.06'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# <h1>PREVENCION DE CANCER</h1>
consejeria_preventiva_cancer1 = curso_vida_adulto(df_data, actividad ='041.Consejería preventiva en factores de riesgo para el cancer 1ra sesion', td=['D'], dx_empiesa_con=[], dx=['99402.08'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_preventiva_cancer2 = curso_vida_adulto(df_data, actividad ='042.Consejería preventiva en factores de riesgo para el cancer 2da sesion', td=['D'], dx_empiesa_con=[], dx=['99402.08'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# <h1>ATENCIÓN SALUD BUCAL</h1>
asb_exam_estomat_NoGestantes_inician = curso_vida_adulto(df_data, actividad ='043.Examen estomatológico no gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D0120'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_exam_estomat_NoGestantes_controlado = curso_vida_adulto(df_data, actividad ='044.Examen estomatológico no gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D0120'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_exam_estomat_gestantes_inician = curso_vida_adulto(df_data, actividad ='045.Examen estomatológico gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D0120'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_exam_estomat_gestantes_controlado = curso_vida_adulto(df_data, actividad ='046.Examen estomatológico gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D0120'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
## Instrucción de Higiene Oral
asb_higieneOral_nogestantes_inician = curso_vida_adulto(df_data, actividad ='047.Instrucción de Higiene Oral No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['D0130'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_higieneOral_nogestantes_controlado = curso_vida_adulto(df_data, actividad ='048.Instrucción de Higiene Oral No Gestantes Controlado', td=['D'], dx_empiesa_con=[], dx=['D0130'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_higieneOral_gestantes_inician = curso_vida_adulto(df_data, actividad ='049.Instrución de Higiene Oral gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D0130'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_higieneOral_gestantes_controlado = curso_vida_adulto(df_data, actividad ='050.Instrución de Higiene Oral gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D0130'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
## Asesoría nutricional para el control de enfermedades dentales
asb_ases_nutri_enf_dent_nogestantes_inician = curso_vida_adulto(df_data, actividad ='051.Asesoría nutricional para el control de enfermedades dentales No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_ases_nutri_enf_dent_nogestantes_controlado = curso_vida_adulto(df_data, actividad ='052.Asesoría nutricional para el control de enfermedades dentales No Gestantes Controlado', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_ases_nutri_enf_dent_gestantes_inician = curso_vida_adulto(df_data, actividad ='053.Asesoría nutricional para el control de enfermedades dentales gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_ases_nutri_enf_dent_gestantes_controlado = curso_vida_adulto(df_data, actividad ='054.Asesoría nutricional para el control de enfermedades dentales gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
## Profilaxis Dental
asb_prof_dent_nogestantes_inician = curso_vida_adulto(df_data, actividad ='055.Profilaxis Dental No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['D1110'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_prof_dent_nogestantes_controlado = curso_vida_adulto(df_data, actividad ='056.Profilaxis Dental No Gestantes Controlado', td=['D'], dx_empiesa_con=[], dx=['D1110'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_prof_dent_gestantes_inician = curso_vida_adulto(df_data, actividad ='057.Profilaxis Dental gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D1110'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_prof_dent_gestantes_controlado = curso_vida_adulto(df_data, actividad ='058.Profilaxis Dental gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D1110'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
#<h3>CONSULTA ESTOMATOLOGICA NO ESPECIALIZADA</h3>
asb_estomatol_noEspec_nogestantes_inician = curso_vida_adulto(df_data, actividad ='059.Consulta Estomatológica no Especializada No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['D0140'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_estomatol_noEspec_nogestantes_controlado = curso_vida_adulto(df_data, actividad ='060.Consulta Estomatológica no Especializada No Gestantes Controlado', td=['D'], dx_empiesa_con=[], dx=['D0140'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_estomatol_noEspec_gestantes_inician = curso_vida_adulto(df_data, actividad ='061.Consulta Estomatológica no Especializada gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D0140'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_estomatol_noEspec_gestantes_controlado = curso_vida_adulto(df_data, actividad ='062.Consulta Estomatológica no Especializada gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D0140'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
### Aplicación Flúor Barniz	
asb_fluor_barniz_nogestantes_inician = curso_vida_adulto(df_data, actividad ='063.Aplicación Fluor Barniz No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['D1206'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_fluor_barniz_nogestantes_controlado = curso_vida_adulto(df_data, actividad ='064.Aplicación Fluor Barniz No Gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D1206'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_fluor_barniz_gestantes_inician = curso_vida_adulto(df_data, actividad ='065.Aplicación Fluor Barniz Gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D1206'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_fluor_barniz_gestantes_controlado = curso_vida_adulto(df_data, actividad ='066.Aplicación Fluor Barniz Gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D1206'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
### RASPAJE DENTA falta arreglar 
asb_raspaje_dent_nogestantes_inician = curso_vida_adulto(df_data, actividad ='067.Raspaje Dental No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['E1311'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_raspaje_dent_nogestantes_controlado = curso_vida_adulto(df_data, actividad ='068.Raspaje Dental No Gestantes Tratado', td=['D'], dx_empiesa_con=[], dx=['E1311'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=['FIN'], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_raspaje_dent_gestantes_inician = curso_vida_adulto(df_data, actividad ='069.Raspaje Dental gestantes inician', td=['D'], dx_empiesa_con=[], dx=['E1311'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_raspaje_dent_gestantes_controlado = curso_vida_adulto(df_data, actividad ='070.Raspaje Dental gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['E1311'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
# SALUD MENTAL 
### Prestaciones de salud mental en adultos
sm_entrev_tamizaje_violencia_fam = curso_vida_adulto(df_data, actividad ='071.Entrevista de tamizaje violencia familiar', td=['D'], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['VIF'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_entrev_tamizaje_violencia_sex = curso_vida_adulto(df_data, actividad ='072.Entrevista de tamizaje violencia sexual', td=['D'], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['VSX'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_entrev_tamizaje_violencia_social = curso_vida_adulto(df_data, actividad ='073.Entrevista de tamizaje violencia social', td=['D'], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['VS'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_entrev_tamizaje_alcoh_drogas = curso_vida_adulto(df_data, actividad ='074.Entrevista de tamizaje alcohol y drogas', td=['D'], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['AD'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_entrev_tamizaje_trans_depres = curso_vida_adulto(df_data, actividad ='075.Entrevista de tamizaje transtornos depresivos', td=['D'], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['TD'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
### tamizaje positivo
sm_tamiz_posit_violen = curso_vida_adulto(df_data, actividad ='076.Problemas relacionados con violencia', td=['D'], dx_empiesa_con=[], dx=['R456'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_tamiz_posit_uso_tab = curso_vida_adulto(df_data, actividad ='077.problemas relacionados con el uso de tabaco', td=['D'], dx_empiesa_con=[], dx=['Z720'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_tamiz_posit_uso_alco = curso_vida_adulto(df_data, actividad ='078.Problemas sociales relacionados con el uso de alcohol', td=['D'], dx_empiesa_con=[], dx=['Z721'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_tamiz_posit_uso_drog = curso_vida_adulto(df_data, actividad ='079.problemas sociales relacionados con el uso de drogas', td=['D'], dx_empiesa_con=[], dx=['Z722'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_tamiz_posit_depre = curso_vida_adulto(df_data, actividad ='080.Depresión', td=[], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['TD'], enclab=[],  td2=['D'], dx2_empiesa_con=[], dx2=['Z133'], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_tamiz_posit_psico = curso_vida_adulto(df_data, actividad ='081.Psicosis', td=[], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['EP'], enclab=[],  td2=['D'], dx2_empiesa_con=[], dx2=['Z133'], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# TBC
### Deteccion de TBC en adultos
tbc_sintom_respir_exam = curso_vida_adulto(df_data, actividad ='082.Sintomaticos respiratorios (TBC)', td=['D','R'], dx_empiesa_con=[], dx=['U200','U2142'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
tbc_sintom_respir_ident = curso_vida_adulto(df_data, actividad ='083.Sintomaticos respiratorios identificados', td=['D'], dx_empiesa_con=[], dx=['U200'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# SALUD SEXUAL Y REPRODUCTIVA
### Prestaciones de Salud Sexual y Reproductiva en Adultos, Gestantes y No gestantes
tamizadas_VIH = curso_vida_adulto(df_data, actividad ='084.Nro de personas tamizadas para VIH', td=[], dx_empiesa_con=[], dx=['Z7172','Z7173'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
tamizadas_VIH_reactivo = curso_vida_adulto(df_data, actividad ='085.Nro de personas tamizadas para VIH con resultado reactivo', td=[], dx_empiesa_con=[], dx=['Z7172'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
atendidas_1erTrimGest = curso_vida_adulto(df_data, actividad ='086.Gestantes atendidas en 1er trimestre de gestación', td=[], dx_empiesa_con=[], dx=['Z3591','Z3491'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
atendidas_2doTrimGest = curso_vida_adulto(df_data, actividad ='087.Gestantes atendidas en 2do trimestre de gestación', td=[], dx_empiesa_con=[], dx=['Z3592','Z3492'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
atendidas_3erTrimGest = curso_vida_adulto(df_data, actividad ='088.Gestantes atendidas en 3er trimestre de gestación', td=[], dx_empiesa_con=[], dx=['Z3593','Z3493'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
gest_controladas = curso_vida_adulto(df_data, actividad ='089.Gestantes controladas (6to control prenatal)', td=[], dx_empiesa_con=[], dx=['Z3491','Z3492','Z3493','Z3591','Z3592','Z3593'], cualquier_dx=[], lab=['6'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
gest_atencion_prenat_reenf = curso_vida_adulto(df_data, actividad ='090.Gestantes con atención prenatal reenfocada', td=['D'], dx_empiesa_con=[], dx=['Z359','Z3591' , 'Z3592' , 'Z3593' , 'Z3491' , 'Z3492' , 'Z3493'], cualquier_dx=[], lab=['6','7','8','9','10'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=['TA'], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
suplement_sulf_ferr_gest = curso_vida_adulto(df_data, actividad ='091.Suplementación con sulfato ferroso en gestantes', td=['D'], dx_empiesa_con=[], dx=['Z359','Z3591' , 'Z3592' , 'Z3593' , 'Z3491' , 'Z3492' , 'Z3493'], cualquier_dx=[], lab=[], enclab=[],  td2=['D'], dx2_empiesa_con=[], dx2=['Z298'], cualquier_dx2=[], lab2=['SF6'], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
suplement_acido_folico_gest = curso_vida_adulto(df_data, actividad ='092.Suplementación con ácido fólico en gestantes', td=['D'], dx_empiesa_con=[], dx=['Z359','Z3591' , 'Z3592' , 'Z3593' , 'Z3491' , 'Z3492' , 'Z3493'], cualquier_dx=[], lab=[], enclab=[],  td2=['D'], dx2_empiesa_con=[], dx2=['Z298'], cualquier_dx2=[], lab2=['AF3'], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
parto = curso_vida_adulto(df_data, actividad ='093.parto', td=['D'], dx_empiesa_con=['O80','O84'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
toma_PAP = curso_vida_adulto(df_data, actividad ='094.Toma de PAP', td=['D'], dx_empiesa_con=[], dx=['Z359'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=['88141','Z0141'], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[],  dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
entrega_result_pap = curso_vida_adulto(df_data, actividad ='095.Entrega de resultados PAP', td=['D'], dx_empiesa_con=[], dx=['U2601'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
result_posit_pap = curso_vida_adulto(df_data, actividad ='096.Resultado positivo de PAP (Gestantes)', td=['D'], dx_empiesa_con=[], dx=['U2601'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=['D060', 'D061', 'D069', 'N879', 'N870', 'N871', 'N872',' Z359'], cualquier_dx2=[], lab2=[], enclab2=['G'], td3=['D'], dx3=['U2601'], dx4=['D060', 'D061', 'D069', 'N879', 'N870', 'N871', 'N872'], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# ITS - VIH
prevencion_ITS_VIH_HB1 = curso_vida_adulto(df_data, actividad ='097.Consejería/Orientación en prevención de ITS,VIH, Hepatitis B 1ra vez', td=[], dx_empiesa_con=[], dx=['99402.05'], cualquier_dx=[], lab=['PV'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
prevencion_ITS_VIH_HB2 = curso_vida_adulto(df_data, actividad ='098.Consejería/Orientación en prevención de ITS,VIH, Hepatitis B, Más de una vez', td=[], dx_empiesa_con=[], dx=['99402.05'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=True)
Consejeria_pretest_vih = curso_vida_adulto(df_data, actividad ='099.Consejería pre-test para VIH', td=['D'], dx_empiesa_con=[], dx=['Z7171'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
Consejeria_postest_posit_vih = curso_vida_adulto(df_data, actividad ='100.Consejería post-test positivo para VIH', td=['D'], dx_empiesa_con=[], dx=['Z7172'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
Consejeria_postest_negat_vih = curso_vida_adulto(df_data, actividad ='101.Consejería post-test negativo para VIH', td=['D'], dx_empiesa_con=[], dx=['Z7173'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
tamizaje_hepatitis_B_reac = curso_vida_adulto(df_data, actividad ='102.Tamizaje en hepatitis B reactivo', td=['D'], dx_empiesa_con=[], dx=['86706'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# SALUD OCULAR
consj_salud_ocular = curso_vida_adulto(df_data, actividad ='103.Consejería en salud ocular', td=['D'], dx_empiesa_con=[], dx=['99401.16'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
det_agud_vis = curso_vida_adulto(df_data, actividad ='104.Determinación de la agudeza visual', td=['D'], dx_empiesa_con=[], dx=['99173'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
limit_vis_moder = curso_vida_adulto(df_data, actividad ='105.Determinación de la agudeza visual', td=['D'], dx_empiesa_con=[], dx=['H456'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
catar_noespefic = curso_vida_adulto(df_data, actividad ='106.Catarata no especificada', td=['D'], dx_empiesa_con=[], dx=['H259'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
exam_ojos = curso_vida_adulto(df_data, actividad ='107.Examen de los ojos y la visión', td=['D'], dx_empiesa_con=[], dx=['Z006'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# ACTIVIDADES PREVENTIVAS PROMOCIONALES
visita_fam_int1 = curso_vida_adulto(df_data, actividad ='108.Visita familiar integral con 1 visita', td=['D'], dx_empiesa_con=[], dx=['99344'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
visita_fam_int2 = curso_vida_adulto(df_data, actividad ='109.Visita familiar integral con 2 visitas', td=['D'], dx_empiesa_con=[], dx=['99344'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
visita_fam_int3 = curso_vida_adulto(df_data, actividad ='110.Visita familiar integral con 3 a más visitas', td=['D'], dx_empiesa_con=[], dx=['99344'], cualquier_dx=[], lab=['3','4','5','6','7'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
vfi_ident_ries_neces = curso_vida_adulto(df_data, actividad ='111.VFI de identificación de riesgos y necesidades', td=[], dx_empiesa_con=[], dx=['C0011.01'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
vfi_cuidados_esenc = curso_vida_adulto(df_data, actividad ='112.VFI para cuidados esenciales', td=[], dx_empiesa_con=[], dx=['C0011.02'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# INMUNIZACIONES
vac_dift1 = curso_vida_adulto(df_data, actividad ='113.Vacunación diftotetánica (dT) no gestantes dosis 1', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['1','D1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
vac_dift2 = curso_vida_adulto(df_data, actividad ='114.Vacunación diftotetánica (dT) no gestantes dosis 2', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['2','D2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
vac_dift3 = curso_vida_adulto(df_data, actividad ='115.Vacunación diftotetánica (dT) no gestantes dosis 3 protegida', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['3','D3'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
vac_dift_G1 = curso_vida_adulto(df_data, actividad ='116.Vacunación diftotetánica (dT) en gestantes dosis 1', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['1','D1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
vac_dift_G2 = curso_vida_adulto(df_data, actividad ='117.Vacunación diftotetánica (dT) en gestantes dosis 2', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['2','D2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
vac_dift_G3 = curso_vida_adulto(df_data, actividad ='118.Vacunación diftotetánica (dT) en gestantes dosis 3 protegida', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['3','D3'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
vac_hepB1 = curso_vida_adulto(df_data, actividad ='119.Vacunación contra la hepatitis B dosis 1', td=['D'], dx_empiesa_con=[], dx=['90744'], cualquier_dx=[], lab=['1','D1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
vac_hepB2 = curso_vida_adulto(df_data, actividad ='120.Vacunación contra la hepatitis B dosis 2', td=['D'], dx_empiesa_con=[], dx=['90744'], cualquier_dx=[], lab=['2','D2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
vac_hepB3 = curso_vida_adulto(df_data, actividad ='121.Vacunación contra la hepatitis B dosis 3 protegido', td=['D'], dx_empiesa_con=[], dx=['90744'], cualquier_dx=[], lab=['3','D3'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
influenzas1 = curso_vida_adulto(df_data, actividad ='122.Influenza dosis 1', td=['D'], dx_empiesa_con=[], dx=['90657'], cualquier_dx=[], lab=['D1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)


C:\Users\IRVIN\AppData\Local\Temp\ipykernel_16128\2930417130.py:191: FutureWarning: The provided callable <function sum at 0x000001FB36208900> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_PAMS = df_PAMS.pivot_table(index=['Anio','Mes'], columns='Genero', values='Nro', aggfunc=np.sum, fill_value=0).reset_index()
C:\Users\IRVIN\AppData\Local\Temp\ipykernel_16128\2930417130.py:195: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(
C:\Users\IRVIN\AppData\Local\Temp\ipykernel_16128\2930417130.py:191: FutureWarning: The provided callable <function sum at 0x000001FB36208900> is currently using DataFrameGroupBy.sum. In a future version of 

In [ ]:
list_adulto = [etsne,obesidad,rinofaringitis_aguda,faringitis_aguda,infecciones_agudas,caries_dental,
               enfermedades_pulpa,gastritis_duodentis,dorsalgia,Otros_transt_sist_uri,plan_atencion_elaborado,
               plan_atencion_ejecutado,imc_delgadezIII,imc_delgadezII,imc_delgadezI,imc_normal,imc_sobrepeso,
               imc_obesidadI,imc_obesidadII,imc_obesidadIII,rn_pab_normal,rn_pab_anormal,
               dnt_val_clinica_sin_fr,dnt_val_clinica_con_fr,dnt_val_clinica_tamizajeLab,
                dxHTA_hipertension_esencial,dxHTA_retinopatia_hipertensiva,dxHTA_nefropatia_hipertensiva,
                dxHTA_diabetes_mellitus_tipo1,dxHTA_diabetes_mellitus_tipo2,dxHTA_diabetes_mellitus_embarazo,
                dxHTA_otras_diabetes_mellitus,dxHTA_nefro_diabetica,dxHTA_retinop_diabetica,
                consejeria_plan_family1,consejeria_plan_family2,consejeria_salud_sexual1,
                consejeria_salud_sexual2,consejeria_perimeno_menopa_climat1,consejeria_perimeno_menopa_climat2,
                consejeria_preventiva_cancer1,consejeria_preventiva_cancer2,asb_exam_estomat_NoGestantes_inician,
                asb_exam_estomat_NoGestantes_controlado,asb_exam_estomat_gestantes_inician,asb_exam_estomat_gestantes_controlado,
                asb_higieneOral_nogestantes_inician,asb_higieneOral_nogestantes_controlado,asb_higieneOral_gestantes_inician,
                asb_higieneOral_gestantes_controlado,asb_ases_nutri_enf_dent_nogestantes_inician,
                asb_ases_nutri_enf_dent_nogestantes_controlado,asb_ases_nutri_enf_dent_gestantes_inician,
                asb_ases_nutri_enf_dent_gestantes_controlado,asb_prof_dent_nogestantes_inician,
                asb_prof_dent_nogestantes_controlado,asb_prof_dent_gestantes_inician,asb_prof_dent_gestantes_controlado,
                asb_estomatol_noEspec_nogestantes_inician,asb_estomatol_noEspec_nogestantes_controlado,asb_estomatol_noEspec_gestantes_inician,
                asb_estomatol_noEspec_gestantes_controlado,
                asb_fluor_barniz_nogestantes_inician,
                asb_fluor_barniz_nogestantes_controlado,asb_fluor_barniz_gestantes_inician,asb_fluor_barniz_gestantes_controlado,
                asb_raspaje_dent_nogestantes_inician,asb_raspaje_dent_nogestantes_controlado,asb_raspaje_dent_gestantes_inician,
                asb_raspaje_dent_gestantes_controlado,sm_entrev_tamizaje_violencia_fam,sm_entrev_tamizaje_violencia_sex,
                sm_entrev_tamizaje_violencia_social,sm_entrev_tamizaje_alcoh_drogas,sm_entrev_tamizaje_trans_depres,
                sm_tamiz_posit_violen,sm_tamiz_posit_uso_tab,sm_tamiz_posit_uso_alco,sm_tamiz_posit_uso_drog,
                sm_tamiz_posit_depre,sm_tamiz_posit_psico,tbc_sintom_respir_exam,tbc_sintom_respir_ident,
                tamizadas_VIH,tamizadas_VIH_reactivo,atendidas_1erTrimGest,atendidas_2doTrimGest,atendidas_3erTrimGest,
                gest_controladas,gest_atencion_prenat_reenf,suplement_sulf_ferr_gest,suplement_acido_folico_gest,parto,
                toma_PAP,entrega_result_pap,result_posit_pap,prevencion_ITS_VIH_HB1,prevencion_ITS_VIH_HB2,Consejeria_pretest_vih,
                Consejeria_postest_posit_vih,Consejeria_postest_negat_vih,tamizaje_hepatitis_B_reac,consj_salud_ocular,det_agud_vis,limit_vis_moder,
                catar_noespefic,exam_ojos,visita_fam_int1,visita_fam_int2,visita_fam_int3,vfi_ident_ries_neces,vfi_cuidados_esenc,
                vac_dift1,vac_dift2,vac_dift3,vac_dift_G1,vac_dift_G2,vac_dift_G3,vac_hepB1,vac_hepB2,vac_hepB3,influenzas1
                ]

In [ ]:
len(list_adulto)

In [ ]:
# concatenamos
df_list_adulto = pd.concat(list_adulto, ignore_index=True)
df_list_adulto

In [ ]:
# genera archivo csv consolidado
df_list_adulto.to_csv('reporte_adulto_consolidado_16122022.csv', index=False, encoding='UTF-8')

# reporte adulto con eess

In [ ]:
# MORBILIDAD DEL ADULTO
etsne = curso_vida_adulto_general(df_data, actividad ='001.Enf. de trans. Sex. no especificada', td=['D'], dx_empiesa_con=['A64'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
obesidad = curso_vida_adulto_general(df_data, actividad ='002.Obesidad', td=['D'], dx_empiesa_con=['E66'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
rinofaringitis_aguda = curso_vida_adulto_general(df_data, actividad ='003.rinofaringitis aguda', td=['D'], dx_empiesa_con=['J00'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
faringitis_aguda = curso_vida_adulto_general(df_data, actividad ='004.Faringitis aguda', td=['D'], dx_empiesa_con=['J02'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
infecciones_agudas = curso_vida_adulto_general(df_data, actividad ='005.Infecciones agudas vias respiratorias superiores', td=['D'], dx_empiesa_con=['J06'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
caries_dental = curso_vida_adulto_general(df_data, actividad ='006.Caries dental', td=['D'], dx_empiesa_con=['K02'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
enfermedades_pulpa = curso_vida_adulto_general(df_data, actividad ='007.Enfer. de la pulpa y de los tejidos periapicales', td=['D'], dx_empiesa_con=['K04'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
gastritis_duodentis = curso_vida_adulto_general(df_data, actividad ='008.Gastritis y duodenitis', td=['D'], dx_empiesa_con=['K29'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dorsalgia = curso_vida_adulto_general(df_data, actividad ='009.Dorsalgia', td=['D'], dx_empiesa_con=['M54'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
Otros_transt_sist_uri = curso_vida_adulto_general(df_data, actividad ='010.Otros transtornos del sistema urinario', td=['D'], dx_empiesa_con=['N39'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# PLAN DE ATENCION INTEGRAL DEL ADULTO
plan_atencion_elaborado = curso_vida_adulto_general(df_data, actividad ='011.Plan de Atención Elaborado', td=[], dx_empiesa_con=[], dx=['C8002'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
plan_atencion_ejecutado = curso_vida_adulto_general(df_data, actividad ='012.Plan de Atención Ejecutado', td=[], dx_empiesa_con=[], dx=['C8002'], cualquier_dx=[], lab=['TA'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# EVALUACIÓN ANTROPOMÉTRICA
imc_delgadezIII = curso_vida_adulto_general(df_data, actividad ='013.Evaluación antropométrica IMC Delgadez III', td=['D'], dx_empiesa_con=[], dx=['E46X3'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_delgadezII = curso_vida_adulto_general(df_data, actividad ='014.Evaluación antropométrica IMC Delgadez II', td=['D'], dx_empiesa_con=[], dx=['E46X2'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_delgadezI = curso_vida_adulto_general(df_data, actividad ='015.Evaluación antropométrica IMC Delgadez I', td=['D'], dx_empiesa_con=[], dx=['E46X1'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_normal = curso_vida_adulto_general(df_data, actividad ='016.Evaluación antropométrica IMC Normal', td=['D'], dx_empiesa_con=[], dx=['Z006'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_sobrepeso = curso_vida_adulto_general(df_data, actividad ='017.Evaluación antropométrica IMC Sobrepeso', td=['D'], dx_empiesa_con=[], dx=['E660'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_obesidadI = curso_vida_adulto_general(df_data, actividad ='018.Evaluación antropométrica IMC Obesidad I', td=['D'], dx_empiesa_con=[], dx=['E669'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_obesidadII = curso_vida_adulto_general(df_data, actividad ='019.Evaluación antropométrica IMC Obesidad II', td=['D'], dx_empiesa_con=[], dx=['E6692'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
imc_obesidadIII = curso_vida_adulto_general(df_data, actividad ='020.Evaluación antropométrica IMC Obesidad III', td=['D'], dx_empiesa_con=[], dx=['E6693'], cualquier_dx=[], lab=['IMC'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# RIESGO NUTRICIONAL
rn_pab_normal = curso_vida_adulto_general(df_data, actividad ='021.Riesgo nutricional PAB Normal', td=['D'], dx_empiesa_con=[], dx=['U8170'], cualquier_dx=[], lab=['N'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
rn_pab_anormal = curso_vida_adulto_general(df_data, actividad ='022.Riesgo nutricional PAB Anormal', td=['D'], dx_empiesa_con=[], dx=['U8170'], cualquier_dx=[], lab=['A'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# <h1> DAÑOS NO TRANSMISIBLE </h1>
dnt_val_clinica_sin_fr = curso_vida_adulto_general(df_data, actividad ='023.Daños No Transmisibles Valoración Clínica Sin Factores de Riesgo', td=['D'], dx_empiesa_con=[], dx=['Z019'], cualquier_dx=[], lab=['DNT'], enclab=[],  td2=['D'], dx2_empiesa_con=[], dx2=['Z006'], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dnt_val_clinica_con_fr = curso_vida_adulto_general(df_data, actividad ='024.Daños No Transmisibles Valoración Clínica con Factores de Riesgo', td=['D'], dx_empiesa_con=[], dx=['Z019'], cualquier_dx=[], lab=['DNT'], enclab=[],  td2=['D'], dx2_empiesa_con=['E66'], dx2=[], cualquier_dx2=['E65X','Z720','Z721','Z723','Z724','Z883','Z834'], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dnt_val_clinica_tamizajeLab = curso_vida_adulto_general(df_data, actividad ='025.Daños No Transmisibles Valoración Clínica y tamizaje de lab.', td=['D'], dx_empiesa_con=[], dx=['U262'], cualquier_dx=[], lab=['DNT'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# casos diagnosticados HTA
dxHTA_hipertension_esencial = curso_vida_adulto_general(df_data, actividad ='026.DX HTA Hipertensión Esencial (primaria)', td=['D'], dx_empiesa_con=[], dx=['I10X'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_retinopatia_hipertensiva = curso_vida_adulto_general(df_data, actividad ='027.DX HTA Retinopatía Hipertensiva', td=['D'], dx_empiesa_con=[], dx=['H350'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_nefropatia_hipertensiva = curso_vida_adulto_general(df_data, actividad ='028.DX HTA Nefropatía Hipertensiva', td=['D'], dx_empiesa_con=[], dx=['I120','I129'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# Casos diagnosticados diabetes mellitus
dxHTA_diabetes_mellitus_tipo1 = curso_vida_adulto_general(df_data, actividad ='029.DX Diabetes Mellitus Tipo 1', td=['D'], dx_empiesa_con=['E10'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_diabetes_mellitus_tipo2 = curso_vida_adulto_general(df_data, actividad ='030.DX Diabetes Mellitus Tipo 2', td=['D'], dx_empiesa_con=['E11'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_diabetes_mellitus_embarazo = curso_vida_adulto_general(df_data, actividad ='031.DX Diabetes Mellitus en el Embarazo', td=['D'], dx_empiesa_con=['O24'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_otras_diabetes_mellitus = curso_vida_adulto_general(df_data, actividad ='032.DX Otras Diabetes Mellitus', td=['D'], dx_empiesa_con=['E12','E13','E14'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_nefro_diabetica = curso_vida_adulto_general(df_data, actividad ='033.DX Nefropatia Diabética', td=['D'], dx_empiesa_con=[], dx=['E102', 'E112', 'E122', 'E132', 'E142'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
dxHTA_retinop_diabetica = curso_vida_adulto_general(df_data, actividad ='034.DX Retinopatia Diabética', td=['D'], dx_empiesa_con=[], dx=['E103', 'E113', 'E123', 'E133', 'E143'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# <h1>PLANIFICACION FAMILIAR</h1>
consejeria_plan_family1 = curso_vida_adulto_general(df_data, actividad ='035.Consejería en planificación familiar 1ra sesion', td=['D'], dx_empiesa_con=[], dx=['99402.04'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_plan_family2 = curso_vida_adulto_general(df_data, actividad ='036.Consejería en planificación familiar 1ra sesion', td=['D'], dx_empiesa_con=[], dx=['99402.04'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_salud_sexual1 = curso_vida_adulto_general(df_data, actividad ='037.Consejería en salud sexual y reproductiva 1ra sesion', td=['D'], dx_empiesa_con=[], dx=['99402.03'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_salud_sexual2 = curso_vida_adulto_general(df_data, actividad ='038.Consejería en salud sexual y reproductiva 2da sesion', td=['D'], dx_empiesa_con=[], dx=['99402.03'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_perimeno_menopa_climat1 = curso_vida_adulto_general(df_data, actividad ='039.Consejería en perimenopausia, menopausia y climaterio 1ra sesion', td=['D'], dx_empiesa_con=[], dx=['99402.06'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_perimeno_menopa_climat2 = curso_vida_adulto_general(df_data, actividad ='040.Consejería en perimenopausia, menopausia y climaterio 2da sesion', td=['D'], dx_empiesa_con=[], dx=['99402.06'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# <h1>PREVENCION DE CANCER</h1>
consejeria_preventiva_cancer1 = curso_vida_adulto_general(df_data, actividad ='041.Consejería preventiva en factores de riesgo para el cancer 1ra sesion', td=['D'], dx_empiesa_con=[], dx=['99402.08'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
consejeria_preventiva_cancer2 = curso_vida_adulto_general(df_data, actividad ='042.Consejería preventiva en factores de riesgo para el cancer 2da sesion', td=['D'], dx_empiesa_con=[], dx=['99402.08'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# <h1>ATENCIÓN SALUD BUCAL</h1>
asb_exam_estomat_NoGestantes_inician = curso_vida_adulto_general(df_data, actividad ='043.Examen estomatológico no gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D0120'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_exam_estomat_NoGestantes_controlado = curso_vida_adulto_general(df_data, actividad ='044.Examen estomatológico no gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D0120'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_exam_estomat_gestantes_inician = curso_vida_adulto_general(df_data, actividad ='045.Examen estomatológico gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D0120'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_exam_estomat_gestantes_controlado = curso_vida_adulto_general(df_data, actividad ='046.Examen estomatológico gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D0120'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
## Instrucción de Higiene Oral
asb_higieneOral_nogestantes_inician = curso_vida_adulto_general(df_data, actividad ='047.Instrucción de Higiene Oral No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['D0130'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_higieneOral_nogestantes_controlado = curso_vida_adulto_general(df_data, actividad ='048.Instrucción de Higiene Oral No Gestantes Controlado', td=['D'], dx_empiesa_con=[], dx=['D0130'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_higieneOral_gestantes_inician = curso_vida_adulto_general(df_data, actividad ='049.Instrución de Higiene Oral gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D0130'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_higieneOral_gestantes_controlado = curso_vida_adulto_general(df_data, actividad ='050.Instrución de Higiene Oral gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D0130'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
## Asesoría nutricional para el control de enfermedades dentales
asb_ases_nutri_enf_dent_nogestantes_inician = curso_vida_adulto_general(df_data, actividad ='051.Asesoría nutricional para el control de enfermedades dentales No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_ases_nutri_enf_dent_nogestantes_controlado = curso_vida_adulto_general(df_data, actividad ='052.Asesoría nutricional para el control de enfermedades dentales No Gestantes Controlado', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_ases_nutri_enf_dent_gestantes_inician = curso_vida_adulto_general(df_data, actividad ='053.Asesoría nutricional para el control de enfermedades dentales gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_ases_nutri_enf_dent_gestantes_controlado = curso_vida_adulto_general(df_data, actividad ='054.Asesoría nutricional para el control de enfermedades dentales gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D1310'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
## Profilaxis Dental
asb_prof_dent_nogestantes_inician = curso_vida_adulto_general(df_data, actividad ='055.Profilaxis Dental No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['D1110'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_prof_dent_nogestantes_controlado = curso_vida_adulto_general(df_data, actividad ='056.Profilaxis Dental No Gestantes Controlado', td=['D'], dx_empiesa_con=[], dx=['D1110'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_prof_dent_gestantes_inician = curso_vida_adulto_general(df_data, actividad ='057.Profilaxis Dental gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D1110'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_prof_dent_gestantes_controlado = curso_vida_adulto_general(df_data, actividad ='058.Profilaxis Dental gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D1110'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
#<h3>CONSULTA ESTOMATOLOGICA NO ESPECIALIZADA</h3>
asb_estomatol_noEspec_nogestantes_inician = curso_vida_adulto_general(df_data, actividad ='059.Consulta Estomatológica no Especializada No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['D0140'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_estomatol_noEspec_nogestantes_controlado = curso_vida_adulto_general(df_data, actividad ='060.Consulta Estomatológica no Especializada No Gestantes Controlado', td=['D'], dx_empiesa_con=[], dx=['D0140'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_estomatol_noEspec_gestantes_inician = curso_vida_adulto_general(df_data, actividad ='061.Consulta Estomatológica no Especializada gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D0140'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_estomatol_noEspec_gestantes_controlado = curso_vida_adulto_general(df_data, actividad ='062.Consulta Estomatológica no Especializada gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D0140'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
### Aplicación Flúor Barniz	
asb_fluor_barniz_nogestantes_inician = curso_vida_adulto_general(df_data, actividad ='063.Aplicación Fluor Barniz No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['D1206'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_fluor_barniz_nogestantes_controlado = curso_vida_adulto_general(df_data, actividad ='064.Aplicación Fluor Barniz No Gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['D1206'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_fluor_barniz_gestantes_inician = curso_vida_adulto_general(df_data, actividad ='065.Aplicación Fluor Barniz Gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D1206'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_fluor_barniz_gestantes_controlado = curso_vida_adulto_general(df_data, actividad ='066.Aplicación Fluor Barniz Gestantes inician', td=['D'], dx_empiesa_con=[], dx=['D1206'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
### RASPAJE DENTA falta arreglar 
asb_raspaje_dent_nogestantes_inician = curso_vida_adulto_general(df_data, actividad ='067.Raspaje Dental No Gestantes Inician', td=['D'], dx_empiesa_con=[], dx=['E1311'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_raspaje_dent_nogestantes_controlado = curso_vida_adulto_general(df_data, actividad ='068.Raspaje Dental No Gestantes Tratado', td=['D'], dx_empiesa_con=[], dx=['E1311'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=['FIN'], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
asb_raspaje_dent_gestantes_inician = curso_vida_adulto_general(df_data, actividad ='069.Raspaje Dental gestantes inician', td=['D'], dx_empiesa_con=[], dx=['E1311'], cualquier_dx=[], lab=['1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
asb_raspaje_dent_gestantes_controlado = curso_vida_adulto_general(df_data, actividad ='070.Raspaje Dental gestantes controlado', td=['D'], dx_empiesa_con=[], dx=['E1311'], cualquier_dx=[], lab=['2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
# SALUD MENTAL 
### Prestaciones de salud mental en adultos
sm_entrev_tamizaje_violencia_fam = curso_vida_adulto_general(df_data, actividad ='071.Entrevista de tamizaje violencia familiar', td=['D'], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['VIF'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_entrev_tamizaje_violencia_sex = curso_vida_adulto_general(df_data, actividad ='072.Entrevista de tamizaje violencia sexual', td=['D'], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['VSX'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_entrev_tamizaje_violencia_social = curso_vida_adulto_general(df_data, actividad ='073.Entrevista de tamizaje violencia social', td=['D'], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['VS'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_entrev_tamizaje_alcoh_drogas = curso_vida_adulto_general(df_data, actividad ='074.Entrevista de tamizaje alcohol y drogas', td=['D'], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['AD'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_entrev_tamizaje_trans_depres = curso_vida_adulto_general(df_data, actividad ='075.Entrevista de tamizaje transtornos depresivos', td=['D'], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['TD'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
### tamizaje positivo
sm_tamiz_posit_violen = curso_vida_adulto_general(df_data, actividad ='076.Problemas relacionados con violencia', td=['D'], dx_empiesa_con=[], dx=['R456'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_tamiz_posit_uso_tab = curso_vida_adulto_general(df_data, actividad ='077.problemas relacionados con el uso de tabaco', td=['D'], dx_empiesa_con=[], dx=['Z720'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_tamiz_posit_uso_alco = curso_vida_adulto_general(df_data, actividad ='078.Problemas sociales relacionados con el uso de alcohol', td=['D'], dx_empiesa_con=[], dx=['Z721'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_tamiz_posit_uso_drog = curso_vida_adulto_general(df_data, actividad ='079.problemas sociales relacionados con el uso de drogas', td=['D'], dx_empiesa_con=[], dx=['Z722'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_tamiz_posit_depre = curso_vida_adulto_general(df_data, actividad ='080.Depresión', td=[], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['TD'], enclab=[],  td2=['D'], dx2_empiesa_con=[], dx2=['Z133'], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
sm_tamiz_posit_psico = curso_vida_adulto_general(df_data, actividad ='081.Psicosis', td=[], dx_empiesa_con=[], dx=['U140'], cualquier_dx=[], lab=['EP'], enclab=[],  td2=['D'], dx2_empiesa_con=[], dx2=['Z133'], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# TBC
### Deteccion de TBC en adultos
tbc_sintom_respir_exam = curso_vida_adulto_general(df_data, actividad ='082.Sintomaticos respiratorios (TBC)', td=['D','R'], dx_empiesa_con=[], dx=['U200','U2142'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
tbc_sintom_respir_ident = curso_vida_adulto_general(df_data, actividad ='083.Sintomaticos respiratorios identificados', td=['D'], dx_empiesa_con=[], dx=['U200'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# SALUD SEXUAL Y REPRODUCTIVA
### Prestaciones de Salud Sexual y Reproductiva en Adultos, Gestantes y No gestantes
tamizadas_VIH = curso_vida_adulto_general(df_data, actividad ='084.Nro de personas tamizadas para VIH', td=[], dx_empiesa_con=[], dx=['Z7172','Z7173'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
tamizadas_VIH_reactivo = curso_vida_adulto_general(df_data, actividad ='085.Nro de personas tamizadas para VIH con resultado reactivo', td=[], dx_empiesa_con=[], dx=['Z7172'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
atendidas_1erTrimGest = curso_vida_adulto_general(df_data, actividad ='086.Gestantes atendidas en 1er trimestre de gestación', td=[], dx_empiesa_con=[], dx=['Z3591','Z3491'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
atendidas_2doTrimGest = curso_vida_adulto_general(df_data, actividad ='087.Gestantes atendidas en 2do trimestre de gestación', td=[], dx_empiesa_con=[], dx=['Z3592','Z3492'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
atendidas_3erTrimGest = curso_vida_adulto_general(df_data, actividad ='088.Gestantes atendidas en 3er trimestre de gestación', td=[], dx_empiesa_con=[], dx=['Z3593','Z3493'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
gest_controladas = curso_vida_adulto_general(df_data, actividad ='089.Gestantes controladas (6to control prenatal)', td=[], dx_empiesa_con=[], dx=['Z3491','Z3492','Z3493','Z3591','Z3592','Z3593'], cualquier_dx=[], lab=['6'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
gest_atencion_prenat_reenf = curso_vida_adulto_general(df_data, actividad ='090.Gestantes con atención prenatal reenfocada', td=['D'], dx_empiesa_con=[], dx=['Z359','Z3591' , 'Z3592' , 'Z3593' , 'Z3491' , 'Z3492' , 'Z3493'], cualquier_dx=[], lab=['6','7','8','9','10'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=['TA'], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
suplement_sulf_ferr_gest = curso_vida_adulto_general(df_data, actividad ='091.Suplementación con sulfato ferroso en gestantes', td=['D'], dx_empiesa_con=[], dx=['Z359','Z3591' , 'Z3592' , 'Z3593' , 'Z3491' , 'Z3492' , 'Z3493'], cualquier_dx=[], lab=[], enclab=[],  td2=['D'], dx2_empiesa_con=[], dx2=['Z298'], cualquier_dx2=[], lab2=['SF6'], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
suplement_acido_folico_gest = curso_vida_adulto_general(df_data, actividad ='092.Suplementación con ácido fólico en gestantes', td=['D'], dx_empiesa_con=[], dx=['Z359','Z3591' , 'Z3592' , 'Z3593' , 'Z3491' , 'Z3492' , 'Z3493'], cualquier_dx=[], lab=[], enclab=[],  td2=['D'], dx2_empiesa_con=[], dx2=['Z298'], cualquier_dx2=[], lab2=['AF3'], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
parto = curso_vida_adulto_general(df_data, actividad ='093.parto', td=['D'], dx_empiesa_con=['O80','O84'], dx=[], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
toma_PAP = curso_vida_adulto_general(df_data, actividad ='094.Toma de PAP', td=['D'], dx_empiesa_con=[], dx=['Z359'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=['88141','Z0141'], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[],  dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
entrega_result_pap = curso_vida_adulto_general(df_data, actividad ='095.Entrega de resultados PAP', td=['D'], dx_empiesa_con=[], dx=['U2601'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
result_posit_pap = curso_vida_adulto_general(df_data, actividad ='096.Resultado positivo de PAP (Gestantes)', td=['D'], dx_empiesa_con=[], dx=['U2601'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=['D060', 'D061', 'D069', 'N879', 'N870', 'N871', 'N872',' Z359'], cualquier_dx2=[], lab2=[], enclab2=['G'], td3=['D'], dx3=['U2601'], dx4=['D060', 'D061', 'D069', 'N879', 'N870', 'N871', 'N872'], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# ITS - VIH
prevencion_ITS_VIH_HB1 = curso_vida_adulto_general(df_data, actividad ='097.Consejería/Orientación en prevención de ITS,VIH, Hepatitis B 1ra vez', td=[], dx_empiesa_con=[], dx=['99402.05'], cualquier_dx=[], lab=['PV'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
prevencion_ITS_VIH_HB2 = curso_vida_adulto_general(df_data, actividad ='098.Consejería/Orientación en prevención de ITS,VIH, Hepatitis B, Más de una vez', td=[], dx_empiesa_con=[], dx=['99402.05'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=True)
Consejeria_pretest_vih = curso_vida_adulto_general(df_data, actividad ='099.Consejería pre-test para VIH', td=['D'], dx_empiesa_con=[], dx=['Z7171'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
Consejeria_postest_posit_vih = curso_vida_adulto_general(df_data, actividad ='100.Consejería post-test positivo para VIH', td=['D'], dx_empiesa_con=[], dx=['Z7172'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
Consejeria_postest_negat_vih = curso_vida_adulto_general(df_data, actividad ='101.Consejería post-test negativo para VIH', td=['D'], dx_empiesa_con=[], dx=['Z7173'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
tamizaje_hepatitis_B_reac = curso_vida_adulto_general(df_data, actividad ='102.Tamizaje en hepatitis B reactivo', td=['D'], dx_empiesa_con=[], dx=['86706'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# SALUD OCULAR
consj_salud_ocular = curso_vida_adulto_general(df_data, actividad ='103.Consejería en salud ocular', td=['D'], dx_empiesa_con=[], dx=['99401.16'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
det_agud_vis = curso_vida_adulto_general(df_data, actividad ='104.Determinación de la agudeza visual', td=['D'], dx_empiesa_con=[], dx=['99173'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
limit_vis_moder = curso_vida_adulto_general(df_data, actividad ='105.Determinación de la agudeza visual', td=['D'], dx_empiesa_con=[], dx=['H456'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
catar_noespefic = curso_vida_adulto_general(df_data, actividad ='106.Catarata no especificada', td=['D'], dx_empiesa_con=[], dx=['H259'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
exam_ojos = curso_vida_adulto_general(df_data, actividad ='107.Examen de los ojos y la visión', td=['D'], dx_empiesa_con=[], dx=['Z006'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# ACTIVIDADES PREVENTIVAS PROMOCIONALES
visita_fam_int1 = curso_vida_adulto_general(df_data, actividad ='108.Visita familiar integral con 1 visita', td=['D'], dx_empiesa_con=[], dx=['99344'], cualquier_dx=[], lab=['1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
visita_fam_int2 = curso_vida_adulto_general(df_data, actividad ='109.Visita familiar integral con 2 visitas', td=['D'], dx_empiesa_con=[], dx=['99344'], cualquier_dx=[], lab=['2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
visita_fam_int3 = curso_vida_adulto_general(df_data, actividad ='110.Visita familiar integral con 3 a más visitas', td=['D'], dx_empiesa_con=[], dx=['99344'], cualquier_dx=[], lab=['3','4','5','6','7'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
vfi_ident_ries_neces = curso_vida_adulto_general(df_data, actividad ='111.VFI de identificación de riesgos y necesidades', td=[], dx_empiesa_con=[], dx=['C0011.01'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
vfi_cuidados_esenc = curso_vida_adulto_general(df_data, actividad ='112.VFI para cuidados esenciales', td=[], dx_empiesa_con=[], dx=['C0011.02'], cualquier_dx=[], lab=[], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
# INMUNIZACIONES
vac_dift1 = curso_vida_adulto_general(df_data, actividad ='113.Vacunación diftotetánica (dT) no gestantes dosis 1', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['1','D1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
vac_dift2 = curso_vida_adulto_general(df_data, actividad ='114.Vacunación diftotetánica (dT) no gestantes dosis 2', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['2','D2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
vac_dift3 = curso_vida_adulto_general(df_data, actividad ='115.Vacunación diftotetánica (dT) no gestantes dosis 3 protegida', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['3','D3'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=['G'], gestantes=False, lab_isnull=False)
vac_dift_G1 = curso_vida_adulto_general(df_data, actividad ='116.Vacunación diftotetánica (dT) en gestantes dosis 1', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['1','D1'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
vac_dift_G2 = curso_vida_adulto_general(df_data, actividad ='117.Vacunación diftotetánica (dT) en gestantes dosis 2', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['2','D2'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
vac_dift_G3 = curso_vida_adulto_general(df_data, actividad ='118.Vacunación diftotetánica (dT) en gestantes dosis 3 protegida', td=['D'], dx_empiesa_con=[], dx=['Z2781'], cualquier_dx=[], lab=['3','D3'], enclab=['G'],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=True, lab_isnull=False)
vac_hepB1 = curso_vida_adulto_general(df_data, actividad ='119.Vacunación contra la hepatitis B dosis 1', td=['D'], dx_empiesa_con=[], dx=['90744'], cualquier_dx=[], lab=['1','D1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
vac_hepB2 = curso_vida_adulto_general(df_data, actividad ='120.Vacunación contra la hepatitis B dosis 2', td=['D'], dx_empiesa_con=[], dx=['90744'], cualquier_dx=[], lab=['2','D2'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
vac_hepB3 = curso_vida_adulto_general(df_data, actividad ='121.Vacunación contra la hepatitis B dosis 3 protegido', td=['D'], dx_empiesa_con=[], dx=['90744'], cualquier_dx=[], lab=['3','D3'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)
influenzas1 = curso_vida_adulto_general(df_data, actividad ='122.Influenza dosis 1', td=['D'], dx_empiesa_con=[], dx=['90657'], cualquier_dx=[], lab=['D1'], enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=False)


In [ ]:
list_adulto_general = [etsne,obesidad,rinofaringitis_aguda,faringitis_aguda,infecciones_agudas,caries_dental,
               enfermedades_pulpa,gastritis_duodentis,dorsalgia,Otros_transt_sist_uri,plan_atencion_elaborado,
               plan_atencion_ejecutado,imc_delgadezIII,imc_delgadezII,imc_delgadezI,imc_normal,imc_sobrepeso,
               imc_obesidadI,imc_obesidadII,imc_obesidadIII,rn_pab_normal,rn_pab_anormal,
               dnt_val_clinica_sin_fr,dnt_val_clinica_con_fr,dnt_val_clinica_tamizajeLab,
                dxHTA_hipertension_esencial,dxHTA_retinopatia_hipertensiva,dxHTA_nefropatia_hipertensiva,
                dxHTA_diabetes_mellitus_tipo1,dxHTA_diabetes_mellitus_tipo2,dxHTA_diabetes_mellitus_embarazo,
                dxHTA_otras_diabetes_mellitus,dxHTA_nefro_diabetica,dxHTA_retinop_diabetica,
                consejeria_plan_family1,consejeria_plan_family2,consejeria_salud_sexual1,
                consejeria_salud_sexual2,consejeria_perimeno_menopa_climat1,consejeria_perimeno_menopa_climat2,
                consejeria_preventiva_cancer1,consejeria_preventiva_cancer2,asb_exam_estomat_NoGestantes_inician,
                asb_exam_estomat_NoGestantes_controlado,asb_exam_estomat_gestantes_inician,asb_exam_estomat_gestantes_controlado,
                asb_higieneOral_nogestantes_inician,asb_higieneOral_nogestantes_controlado,asb_higieneOral_gestantes_inician,
                asb_higieneOral_gestantes_controlado,asb_ases_nutri_enf_dent_nogestantes_inician,
                asb_ases_nutri_enf_dent_nogestantes_controlado,asb_ases_nutri_enf_dent_gestantes_inician,
                asb_ases_nutri_enf_dent_gestantes_controlado,asb_prof_dent_nogestantes_inician,
                asb_prof_dent_nogestantes_controlado,asb_prof_dent_gestantes_inician,asb_prof_dent_gestantes_controlado,
                asb_estomatol_noEspec_nogestantes_inician,asb_estomatol_noEspec_nogestantes_controlado,asb_estomatol_noEspec_gestantes_inician,
                asb_estomatol_noEspec_gestantes_controlado,
                asb_fluor_barniz_nogestantes_inician,
                asb_fluor_barniz_nogestantes_controlado,asb_fluor_barniz_gestantes_inician,asb_fluor_barniz_gestantes_controlado,
                asb_raspaje_dent_nogestantes_inician,asb_raspaje_dent_nogestantes_controlado,asb_raspaje_dent_gestantes_inician,
                asb_raspaje_dent_gestantes_controlado,sm_entrev_tamizaje_violencia_fam,sm_entrev_tamizaje_violencia_sex,
                sm_entrev_tamizaje_violencia_social,sm_entrev_tamizaje_alcoh_drogas,sm_entrev_tamizaje_trans_depres,
                sm_tamiz_posit_violen,sm_tamiz_posit_uso_tab,sm_tamiz_posit_uso_alco,sm_tamiz_posit_uso_drog,
                sm_tamiz_posit_depre,sm_tamiz_posit_psico,tbc_sintom_respir_exam,tbc_sintom_respir_ident,
                tamizadas_VIH,tamizadas_VIH_reactivo,atendidas_1erTrimGest,atendidas_2doTrimGest,atendidas_3erTrimGest,
                gest_controladas,gest_atencion_prenat_reenf,suplement_sulf_ferr_gest,suplement_acido_folico_gest,parto,
                toma_PAP,entrega_result_pap,result_posit_pap,prevencion_ITS_VIH_HB1,prevencion_ITS_VIH_HB2,Consejeria_pretest_vih,
                Consejeria_postest_posit_vih,Consejeria_postest_negat_vih,tamizaje_hepatitis_B_reac,consj_salud_ocular,det_agud_vis,limit_vis_moder,
                catar_noespefic,exam_ojos,visita_fam_int1,visita_fam_int2,visita_fam_int3,vfi_ident_ries_neces,vfi_cuidados_esenc,
                vac_dift1,vac_dift2,vac_dift3,vac_dift_G1,vac_dift_G2,vac_dift_G3,vac_hepB1,vac_hepB2,vac_hepB3,influenzas1
                ]

In [ ]:
# concatenamos
df_list_adulto_general = pd.concat(list_adulto_general, ignore_index=True)
df_list_adulto_general

In [ ]:
# leer establecimientos de salud
df_eess = pd.read_csv(ruta_datos+'/Hisminsa_2022/establecimientos/eess.csv', engine='python', encoding='ISO-8859-1')
df_eess

In [ ]:
# union de diagnosticos, pacientes y eess
df_reporte_adulto_general = pd.merge(df_list_adulto_general,df_eess, left_on='Id_Establecimiento', right_on='Id_eess', how='inner')
df_reporte_adulto_general

In [ ]:
df_reporte_adulto_general = df_reporte_adulto_general[['Sector','Dpto','Provincia','Distrito','Desc_UE','Red', 'MicroRed','Nombre_eess', 'Cat','Actividad','Anio', 'Mes', 'Masculino','Femenino', 'Total']]
df_reporte_adulto_general

In [ ]:
df_reporte_adulto_general = df_reporte_adulto_general[df_reporte_adulto_general.Mes>=3]
df_reporte_adulto_general

In [ ]:
# genera archivo csv general
df_reporte_adulto_general.to_csv('reporte_adulto_general_16122022_03_12.csv', index=False, encoding='UTF-8')

<h1>Metodo de planificación familiar en Adultos</h1>

In [ ]:
# Funcion para calcular el total por genero y por grupo etareo de las actividades de etapa de vida adulto mayor
def curso_vida_adulto_plani_fam(df,**kwargs):
    # Parametros
    act = kwargs['actividad']
    dx_empiesa_con = kwargs['dx_empiesa_con']
    dx2_empiesa_con = kwargs['dx2_empiesa_con']
    dx = kwargs['dx']
    lab = kwargs['lab']
    genero = kwargs['genero']
    lab2 = kwargs['lab2']
    dx2 = kwargs['dx2']
    dx3 = kwargs['dx3']
    dx4 = kwargs['dx4']
    td = kwargs['td']
    td2 = kwargs['td2']
    td3 = kwargs['td3']
    enclab = kwargs['enclab']
    enclab2 = kwargs['enclab2']
    enclab_menos_en = kwargs['enclab_menos_en']
    cualquier_dx2 = kwargs['cualquier_dx2']
    gestantes = kwargs['gestantes']
    lab_isnull = kwargs['lab_isnull']
    #grupos etareos adultero mayor
    #list_ge = [[30,59],[30,59]]
    # lista de dataframes resultados segun grupo etareo
    #list=[]
    # Filtroltos por edad etarea, dx, y labs
    #for i in list_ge:
    df_GE = df[df['Anio_Actual_Paciente'].between(30,59)]
    if td:
        if lab and genero:
            # if genero:
            df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx)) & ((df_GE.Valor_Lab.isin(lab))) & ((df_GE.Genero.isin(genero)))]
            #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita'])
            # if gestantes:
            #     if enclab:
            #         df_geB = df_GE[(df_GE.Genero=='F') | (df_GE.Valor_Lab.isin(enclab))]
            #         df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
            #         df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente']) 
                    
            #     if lab2:
            #         df_geB = df_GE[(df_GE.Valor_Lab.isin(lab2))]
            #         df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
            #         #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
            #     # if enclab_menos_en:
            #     #     df_geB = df_GE[(df_GE.Genero=='F') | (~(df_GE.Valor_Lab.isin(enclab_menos_en)))]
            #     #     df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
            #     #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
            # else: # no gestantes
            #     if enclab_menos_en:
            #         # df_x = df_GE[(df_GE.Genero=='F') | (df_GE.Valor_Lab.isin(enclab_menos_en))].index
            #         # df_y = pd.merge(df_PAMS,df_x, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
            #         # df_xy = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                   
            #         df_geB = df_GE[(~(df_GE.Valor_Lab.isin(enclab_menos_en)))]
            #         df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
            #         df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                    
            #         # df_PAMS = pd.merge(df_PAMS,df_xy, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner')
            #         # df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
                    
            #         # df_PAMS = df_PAMS.drop(df_xy.index)
                    
            #         #df_PAMS = df_PAMS[df_PAMS.Id_Ups_x==303304]
            #         #df_PAMS = df_PAMS[df_PAMS.Id_Profesion_x.isin([25,47])]  
            #     if enclab:
            #         df_geB = df_GE[(df_GE.Valor_Lab.isin(enclab))]
            #         df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
            #         #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
            #     if lab2:
            #         df_geB = df_GE[(df_GE.Valor_Lab.isin(lab2))]
            #         df_PAMS = pd.merge(df_PAMS,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
            #         #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
        elif enclab:
            df_geA = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx))]
            if enclab_menos_en:
                df_geB = df_GE[~(df_GE.Valor_Lab.isin(enclab_menos_en))] #ojo se modifico enclab
                df_PAMS = pd.merge(df_geA,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Codigo_Item','Valor_Lab','Id_Correlativo','Id_Correlativo_Lab'])
                df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
            else:
                df_geB = df_GE[(df_GE.Valor_Lab.isin(enclab))]
                df_PAMS = pd.merge(df_geA,df_geB, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'])
                df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])  
        elif dx_empiesa_con:
            list_tmp=[]
            for k in dx_empiesa_con:
                df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.str.startswith(k, na=False))]
                list_tmp.append(df_PAMS)
            if len(list_tmp)==0:
                #df_PAMS = pd.DataFrame(columns=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Nro'], index=range(1))
                df_PAMS.empty = True
            elif len(list_tmp)==1:
                df_PAMS = list_tmp[0]
            elif len(list_tmp)==2:
                df_PAMS = pd.merge(list_tmp[0],list_tmp[1], on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='outer')
            elif len(list_tmp)==3:
                df_PAMS = list_tmp[0].merge(list_tmp[1], on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='inner').merge(list_tmp[2], on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='outer')
                #df_PAMS = pd.merge(df_PAMS,list_tmp[j+1], on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'])
                # df_PAMS = pd.concat(list_tmp, ignore_index=True)
            #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])  
        elif lab_isnull and genero:
            df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx)) & (df_GE.Valor_Lab.isnull()) & (df_GE.Genero.isin(genero))] 
        elif genero:
            df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx)) & (df_GE.Genero.isin(genero))] 
        else:
            #df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.str.startswith(dx[0], na=False))]
            df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx))]
            #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita'])
            # if cualquier_cod:
            #     w = df_GE[(df_GE.Codigo_Item.isin(dx))]
            #     y = pd.merge(df_PAMS,w, on=['Id_Cita','Anio','Mes','Genero'], how='left')
            #     d1=df_GE[(df_GE.Codigo_Item.isin(dx))]
            #     d2=df_GE[(df_GE.Codigo_Item.isin(['99401']))]
            #     d = pd.merge(d1,d2, on=['Id_Cita','Anio','Mes','Genero'])
            #     df_gem = pd.merge(y,d, on=['Id_Cita','Anio','Mes','Genero'], how='left')
            #     df_PAMS = df_gem.drop_duplicates(subset=['Id_Cita'])
    elif lab:
        df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx)) & (df_GE.Valor_Lab.isin(lab))]
    elif lab_isnull:
        df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx)) & (df_GE.Valor_Lab.isnull())]  
    else:
        df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx))]
    # df_bl = df_GE[(df_GE.Valor_Lab.isnull())]
    # df_PAMS = pd.merge(df_PAMS1,df_bl, on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'],how='inner')
        

    # if dx2:
    #     df_PAMS1 = df_GE[df_GE.Codigo_Item.isin(dx2)]
    #     df_PAMS = pd.merge(df_PAMS, df_PAMS1, on=["Id_Cita",'Anio','Mes','Genero'], how='inner')
        
    # if td2 and dx2 and lab2:
    #     df_PAMS2 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.isin(dx2)) & ((df_GE.Valor_Lab.isin(lab2)))]
    #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita'])
    # elif td2 and dx2:
    #     df_PAMS2 = df_GE[(df_GE.Tipo_Diagnostico.isin(td2)) & (df_GE.Codigo_Item.isin(dx2))]  
    #     df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita']) 

    # if td:
    #     df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td))]
    #     df_PAMS['Valor_Lab'] = df_PAMS['Valor_Lab'].str.extract('(\d+(?:\.\d+)?)')
    #     df_PAMS = df_PAMS[df_PAMS.Codigo_Item.isin(dx)]
        
    # if dx:
    #     df_PAMS = df_GE[df_GE.Codigo_Item.isin(dx)]
    
    if not (df_PAMS.empty): #activos nuevos
        df_PAMS['Valor_Lab'] = df_PAMS['Valor_Lab'].fillna(0)
        df_PAMS = df_PAMS.astype({'Valor_Lab':'float64'})
        #df_PAMS = df_PAMS.reset_index()
        # agrupamos y contamos por genero
        df_PAMS = df_PAMS.groupby(['Anio','Mes'])['Valor_Lab'].agg(['count','sum']).reset_index()
        df_PAMS = df_PAMS.rename({'count':'Activ','sum':'Insumo'}, axis=1)
        # df_PAMS = df_PAMS.groupby(['Anio','Mes'])['Valor_Lab'].agg(['count']).reset_index()
        # df_PAMS = df_PAMS.rename({'count':'Activ'}, axis=1)
        # transponemos genero
        # df_PAMS = df_PAMS.pivot_table(index=['Anio','Mes'], columns='Genero', values='Nro', aggfunc=np.sum, fill_value=0).reset_index()
        # if not (df_PAMS.empty):
        #     if set(['M','F']).issubset(df_PAMS.columns):
        #         # suma de los generos por meses
        #         df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(
        #             Masculino = ('M', sum),
        #             Femenino = ('F', sum)
        #         ).reset_index()
        #     elif 'M' in df_PAMS.columns:
        #         # suma solo masculinos
        #         df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(
        #             Masculino = ('M', sum),
        #         ).reset_index()
        #         df_PAMS = df_PAMS.assign(Femenino=0)
        #         #print(df_PAMS)
        #     else:
        #         # suma solo femeninos
        #         df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(
        #             Femenino = ('F', sum)
        #         ).reset_index()
        #         df_PAMS = df_PAMS.assign(Masculino=0)
        #         df_PAMS = df_PAMS[['Anio','Mes','Masculino','Femenino']]
        #     #list.append(df_PAMS)
        # else:
        #     df_PAMS = pd.DataFrame(columns=['Anio','Mes','Masculino','Femenino'], index=range(1))
        #list.append(df_PAMS)
    else:
        df_PAMS = pd.DataFrame(columns=['Anio','Mes','Activ','Insumo'], index=range(1))
        
    # if not (df_PAMS2.empty): #activos continuadores
        # df_PAMS2['Valor_Lab'] = df_PAMS2['Valor_Lab'].fillna(0)
        # df_PAMS2 = df_PAMS2.astype({'Valor_Lab':'float64'})
        #df_PAMS = df_PAMS.reset_index()
        # agrupamos y contamos por genero
        # df_PAMS2 = df_PAMS2.groupby(['Anio','Mes'])['Valor_Lab'].agg(['count','sum']).reset_index()
        # df_PAMS2 = df_PAMS2.rename({'count':'Activ','sum':'Insumo'}, axis=1)
        # df_PAMS2 = df_PAMS2.groupby(['Anio','Mes'])['Valor_Lab'].agg(['count']).reset_index()
        # df_PAMS2 = df_PAMS2.rename({'count':'Activ'}, axis=1)
        # transponemos genero
        # df_PAMS = df_PAMS.pivot_table(index=['Anio','Mes'], columns='Genero', values='Nro', aggfunc=np.sum, fill_value=0).reset_index()
        # if not (df_PAMS.empty):
        #     if set(['M','F']).issubset(df_PAMS.columns):
        #         # suma de los generos por meses
        #         df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(
        #             Masculino = ('M', sum),
        #             Femenino = ('F', sum)
        #         ).reset_index()
        #     elif 'M' in df_PAMS.columns:
        #         # suma solo masculinos
        #         df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(
        #             Masculino = ('M', sum),
        #         ).reset_index()
        #         df_PAMS = df_PAMS.assign(Femenino=0)
        #         #print(df_PAMS)
        #     else:
        #         # suma solo femeninos
        #         df_PAMS=df_PAMS.groupby(['Anio','Mes']).agg(
        #             Femenino = ('F', sum)
        #         ).reset_index()
        #         df_PAMS = df_PAMS.assign(Masculino=0)
        #         df_PAMS = df_PAMS[['Anio','Mes','Masculino','Femenino']]
        #     #list.append(df_PAMS)
        # else:
        #     df_PAMS = pd.DataFrame(columns=['Anio','Mes','Masculino','Femenino'], index=range(1))
        #list.append(df_PAMS)
    # else:
    #     df_PAMS2 = pd.DataFrame(columns=['Anio','Mes','Activ'], index=range(1))    
    
    #list.append(df_PAMS)   

    # df_PAMS = pd.merge(df_PAMS, df_PAMS2, on=["Anio","Mes"],how='outer',suffixes=("_Nuevas","_Continuadoras")).fillna(0)
    # df_PAMS = pd.merge(df_PAMS,df_PAMS2, on=["Anio","Mes"],how='outer',suffixes=("_Nuevas","_Continuadoras")).fillna(0)
    # df_PAMS = pd.merge(df_PAMS, list[2], on=["Anio","Mes"],how='outer').fillna(0)
    # if set(['Masculino_30_59','Masculino_30_59','Femenino_30_59','Femenino_30_59']).issubset(df_PAMS.columns):
    #     df_PAMS['Total_Masculinos'] = df_PAMS['Masculino_60_69']+df_PAMS['Masculino_70_79']+df_PAMS['Masculino']
    #     df_PAMS['Total_Femeninos'] = df_PAMS['Femenino_60_69']+df_PAMS['Femenino_70_79']+df_PAMS['Femenino']
    # elif set(['Masculino_60_69','Masculino_70_79','Masculino']).issubset(df_PAMS.columns):
    #     df_PAMS['Total_Masculinos'] = df_PAMS['Masculino_60_69']+df_PAMS['Masculino_70_79']+df_PAMS['Masculino']
    # else:
    
    #df_PAMS['Total'] = df_PAMS['Masculino']+df_PAMS['Femenino']
    df_PAMS = df_PAMS[~df_PAMS.Mes.isnull()]
    # df_PAMS = df_PAMS.astype(int)
    # df_PAMS =  df_PAMS.sort_values(['Anio','Mes'])
    df_PAMS = df_PAMS.assign(Actividad=act)
    # df_PAMS = df_PAMS[df_PAMS.Mes!=0]

    return df_PAMS

In [ ]:
# Funcion para calcular el total por genero y por grupo etareo de las actividades de etapa de vida adulto mayor
def curso_vida_adulto_plani_fam_eess(df,**kwargs):
    # Parametros
    act = kwargs['actividad']
    dx_empiesa_con = kwargs['dx_empiesa_con']
    dx2_empiesa_con = kwargs['dx2_empiesa_con']
    dx = kwargs['dx']
    lab = kwargs['lab']
    genero = kwargs['genero']
    lab2 = kwargs['lab2']
    dx2 = kwargs['dx2']
    dx3 = kwargs['dx3']
    dx4 = kwargs['dx4']
    td = kwargs['td']
    td2 = kwargs['td2']
    td3 = kwargs['td3']
    enclab = kwargs['enclab']
    enclab2 = kwargs['enclab2']
    enclab_menos_en = kwargs['enclab_menos_en']
    cualquier_dx2 = kwargs['cualquier_dx2']
    gestantes = kwargs['gestantes']
    lab_isnull = kwargs['lab_isnull']
  
    df_GE = df[df['Anio_Actual_Paciente'].between(30,59)]
    if td:
        if lab and genero:
            # if genero:
            df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx)) & ((df_GE.Valor_Lab.isin(lab))) & ((df_GE.Genero.isin(genero)))]

        elif enclab:
            df_geA = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx))]
            if enclab_menos_en:
                df_geB = df_GE[~(df_GE.Valor_Lab.isin(enclab_menos_en))] #ojo se modifico enclab
                df_PAMS = pd.merge(df_geA,df_geB, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero','Codigo_Item','Valor_Lab','Id_Correlativo','Id_Correlativo_Lab'])
                df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])
            else:
                df_geB = df_GE[(df_GE.Valor_Lab.isin(enclab))]
                df_PAMS = pd.merge(df_geA,df_geB, on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'])
                df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita','Id_Paciente'])  
        elif dx_empiesa_con:
            list_tmp=[]
            for k in dx_empiesa_con:
                df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.str.startswith(k, na=False))]
                list_tmp.append(df_PAMS)
            if len(list_tmp)==0:
                #df_PAMS = pd.DataFrame(columns=['Id_Cita','Id_Paciente','Anio','Mes','Genero','Nro'], index=range(1))
                df_PAMS.empty = True
            elif len(list_tmp)==1:
                df_PAMS = list_tmp[0]
            elif len(list_tmp)==2:
                df_PAMS = pd.merge(list_tmp[0],list_tmp[1], on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='outer')
            elif len(list_tmp)==3:
                df_PAMS = list_tmp[0].merge(list_tmp[1], on=['Id_Cita','Id_Paciente','Id_Establecimiento','Anio','Mes','Genero'], how='inner').merge(list_tmp[2], on=['Id_Cita','Id_Paciente','Anio','Mes','Genero'], how='outer')
  
        elif lab_isnull and genero:
            df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx)) & (df_GE.Valor_Lab.isnull()) & (df_GE.Genero.isin(genero))] 
        elif genero:
            df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx)) & (df_GE.Genero.isin(genero))] 
        else:
            #df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.str.startswith(dx[0], na=False))]
            df_PAMS = df_GE[(df_GE.Tipo_Diagnostico.isin(td)) & (df_GE.Codigo_Item.isin(dx))]
            #df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita'])

    elif lab:
        df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx)) & (df_GE.Valor_Lab.isin(lab))]
    elif lab_isnull:
        df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx)) & (df_GE.Valor_Lab.isnull())]  
    else:
        df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx))]

    
    if not (df_PAMS.empty): #activos nuevos
        df_PAMS['Valor_Lab'] = df_PAMS['Valor_Lab'].fillna(0)
        df_PAMS = df_PAMS.astype({'Valor_Lab':'float64'})
        #df_PAMS = df_PAMS.reset_index()
        # agrupamos y contamos por genero
        df_PAMS = df_PAMS.groupby(['Anio','Mes','Id_Establecimiento'])['Valor_Lab'].agg(['count','sum']).reset_index()
        df_PAMS = df_PAMS.rename({'count':'Activ','sum':'Insumo'}, axis=1)
    else:
        df_PAMS = pd.DataFrame(columns=['Anio','Mes','Activ','Insumo','Id_Establecimiento'], index=range(1))
        
    #df_PAMS['Total'] = df_PAMS['Masculino']+df_PAMS['Femenino']
    df_PAMS = df_PAMS[~df_PAMS.Mes.isnull()]
    # df_PAMS = df_PAMS.astype(int)
    # df_PAMS =  df_PAMS.sort_values(['Anio','Mes'])
    df_PAMS = df_PAMS.assign(Actividad=act)
    # df_PAMS = df_PAMS[df_PAMS.Mes!=0]

    return df_PAMS

In [ ]:
def metodo_planif_fam_adulto(df_data,**kwargs):
    act = kwargs['actividad']
    td_n = kwargs['td_n']
    dx_n = kwargs['dx_n']
    lab_n = kwargs['lab_n']
    genero_n = kwargs['genero_n']
    lab_isnull_n = kwargs['lab_isnull_n']
    
    td_c = kwargs['td_c']
    dx_c = kwargs['dx_c']
    lab_c = kwargs['lab_c']
    genero_c = kwargs['genero_c']
    lab_isnull_c = kwargs['lab_isnull_c']
    
    x = curso_vida_adulto_plani_fam(df_data, actividad = act,  td=td_n, dx_empiesa_con=[], dx=dx_n, cualquier_dx=[], lab=lab_n, enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=lab_isnull_n, genero=genero_n)
    y = curso_vida_adulto_plani_fam(df_data, actividad = act,  td=td_c, dx_empiesa_con=[], dx=dx_c, cualquier_dx=[], lab=lab_c, enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=lab_isnull_c, genero=genero_c)
    w = pd.merge(x,y, on=['Anio','Mes','Actividad'], how='outer',suffixes=("_Nuevas","_Continuadoras")).fillna(0)
    
    return w  

In [ ]:
def metodo_planif_fam_adulto_eess(df,**kwargs):
    act = kwargs['actividad']
    td_n = kwargs['td_n']
    dx_n = kwargs['dx_n']
    lab_n = kwargs['lab_n']
    genero_n = kwargs['genero_n']
    lab_isnull_n = kwargs['lab_isnull_n']
    
    td_c = kwargs['td_c']
    dx_c = kwargs['dx_c']
    lab_c = kwargs['lab_c']
    genero_c = kwargs['genero_c']
    lab_isnull_c = kwargs['lab_isnull_c']
    
    x = curso_vida_adulto_plani_fam_eess(df, actividad = act,  td=td_n, dx_empiesa_con=[], dx=dx_n, cualquier_dx=[], lab=lab_n, enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=lab_isnull_n, genero=genero_n)
    y = curso_vida_adulto_plani_fam_eess(df, actividad = act,  td=td_c, dx_empiesa_con=[], dx=dx_c, cualquier_dx=[], lab=lab_c, enclab=[],  td2=[], dx2_empiesa_con=[], dx2=[], cualquier_dx2=[], lab2=[], enclab2=[], td3=[], dx3=[], dx4=[], enclab_menos_en=[], gestantes=False, lab_isnull=lab_isnull_c, genero=genero_c)
    w = pd.merge(x,y, on=['Anio','Mes','Actividad','Id_Establecimiento'], how='outer',suffixes=("_Nuevas","_Continuadoras")).fillna(0)
    
    return w


In [ ]:
a = metodo_planif_fam_adulto(df_data, actividad ='01.DIU',  td_n=['D'], dx_n=['58300'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z305'], lab_c=[], lab_isnull_c=False, genero_c=[])
b = metodo_planif_fam_adulto(df_data, actividad ='02.Hormonal Oral Combinado',  td_n=['D'], dx_n=['Z3003'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z3043'], lab_c=[], lab_isnull_c=False, genero_c=[])
c = metodo_planif_fam_adulto(df_data, actividad ='03.Hormonal Inyectable Trimestral',  td_n=['D'], dx_n=['Z30052'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30452'], lab_c=[], lab_isnull_c=False, genero_c=[])
d = metodo_planif_fam_adulto(df_data, actividad ='04.Hormonal Inyectable Mensual', td_n=['D'], dx_n=['Z30051'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30451'], lab_c=[], lab_isnull_c=False, genero_c=[])
e = metodo_planif_fam_adulto(df_data, actividad ='05.Hormonal Implante', td_n=['D'], dx_n=['Z3006'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z3046'], lab_c=[], lab_isnull_c=False, genero_c=[])
f = metodo_planif_fam_adulto(df_data, actividad ='06.Barrera Condon Masculino', td_n=['D'], dx_n=['Z3008'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z3048'], lab_c=[], lab_isnull_c=False, genero_c=[])
g = metodo_planif_fam_adulto(df_data, actividad ='07.Barrera Condon Femenino', td_n=['D'], dx_n=['Z3009'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z3049'], lab_c=[], lab_isnull_c=False, genero_c=[])
h = metodo_planif_fam_adulto(df_data, actividad ='08.AQV Femenino', td_n=['D'], dx_n=['Z302'], lab_n=[], lab_isnull_n=True, genero_n=['F'], td_c=['D'], dx_c=['Z302'], lab_c=['1'], lab_isnull_c=False, genero_c=['F'])
i = metodo_planif_fam_adulto(df_data, actividad ='09.AQV Masculino', td_n=['D'], dx_n=['Z302'], lab_n=[], lab_isnull_n=True, genero_n=['M'], td_c=['D'], dx_c=['Z302'], lab_c=['1'], lab_isnull_c=False, genero_c=['F'])
j = metodo_planif_fam_adulto(df_data, actividad ='10.MELA', td_n=['D'], dx_n=['Z30091'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30491'], lab_c=[], lab_isnull_c=False, genero_c=[])
k = metodo_planif_fam_adulto(df_data, actividad ='11.Abstinencia Periodica Billings', td_n=['D'], dx_n=['Z30093'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30493'], lab_c=[], lab_isnull_c=False, genero_c=[])
l = metodo_planif_fam_adulto(df_data, actividad ='12.Abstinencia Periodica Ritmo', td_n=['D'], dx_n=['Z30092'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30492'], lab_c=[], lab_isnull_c=False, genero_c=[])
m = metodo_planif_fam_adulto(df_data, actividad ='13.Abstinencia Periodica Días Fijos', td_n=['D'], dx_n=['Z30094'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30494'], lab_c=[], lab_isnull_c=False, genero_c=[])
n = metodo_planif_fam_adulto(df_data, actividad ='14.Anticoncepción Oral de Emergencia/YUZPE', td_n=['D'], dx_n=['Z3091'], lab_n=[], lab_isnull_n=False, genero_n=['F'], td_c=['D'], dx_c=['Z3091'], lab_c=[], lab_isnull_c=False, genero_c=['F'])
o = metodo_planif_fam_adulto(df_data, actividad ='15.Anticoncepción Oral de Emergencia/Progestageno', td_n=['D'], dx_n=['Z3092'], lab_n=[], lab_isnull_n=False, genero_n=['F'], td_c=['D'], dx_c=['Z3092'], lab_c=[], lab_isnull_c=False, genero_c=['F'])

In [ ]:
list_planFam_consolidado = [a,b,c,d,e,f,g,h,i,j,k,l,m,n,o]

In [ ]:
# concatenamos
df_list_planFam_consolidado = pd.concat(list_planFam_consolidado, ignore_index=True)
df_list_planFam_consolidado

In [ ]:
# genera archivo csv general
df_list_planFam_consolidado.to_csv('reporte_adulto_planFam_consolidado_16122022.csv', index=False, encoding='UTF-8')

In [ ]:
# eliminamos variables 
del a,b,c,d,e,f,g,h,i,j,k,l,m,n,o
del list_planFam_consolidado
del df_list_planFam_consolidado

# planificacion familiar con eess

In [ ]:
ba = metodo_planif_fam_adulto_eess(df_data, actividad ='01.DIU',  td_n=['D'], dx_n=['58300'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z305'], lab_c=[], lab_isnull_c=False, genero_c=[])
bb = metodo_planif_fam_adulto_eess(df_data, actividad ='02.Hormonal Oral Combinado',  td_n=['D'], dx_n=['Z3003'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z3043'], lab_c=[], lab_isnull_c=False, genero_c=[])
bc = metodo_planif_fam_adulto_eess(df_data, actividad ='03.Hormonal Inyectable Trimestral',  td_n=['D'], dx_n=['Z30052'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30452'], lab_c=[], lab_isnull_c=False, genero_c=[])
bd = metodo_planif_fam_adulto_eess(df_data, actividad ='04.Hormonal Inyectable Mensual', td_n=['D'], dx_n=['Z30051'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30451'], lab_c=[], lab_isnull_c=False, genero_c=[])
be = metodo_planif_fam_adulto_eess(df_data, actividad ='05.Hormonal Implante', td_n=['D'], dx_n=['Z3006'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z3046'], lab_c=[], lab_isnull_c=False, genero_c=[])
bf = metodo_planif_fam_adulto_eess(df_data, actividad ='06.Barrera Condon Masculino', td_n=['D'], dx_n=['Z3008'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z3048'], lab_c=[], lab_isnull_c=False, genero_c=[])
bg = metodo_planif_fam_adulto_eess(df_data, actividad ='07.Barrera Condon Femenino', td_n=['D'], dx_n=['Z3009'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z3049'], lab_c=[], lab_isnull_c=False, genero_c=[])
bh = metodo_planif_fam_adulto_eess(df_data, actividad ='08.AQV Femenino', td_n=['D'], dx_n=['Z302'], lab_n=[], lab_isnull_n=True, genero_n=['F'], td_c=['D'], dx_c=['Z302'], lab_c=['1'], lab_isnull_c=False, genero_c=['F'])
bi = metodo_planif_fam_adulto_eess(df_data, actividad ='09.AQV Masculino', td_n=['D'], dx_n=['Z302'], lab_n=[], lab_isnull_n=True, genero_n=['M'], td_c=['D'], dx_c=['Z302'], lab_c=['1'], lab_isnull_c=False, genero_c=['F'])
bj = metodo_planif_fam_adulto_eess(df_data, actividad ='10.MELA', td_n=['D'], dx_n=['Z30091'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30491'], lab_c=[], lab_isnull_c=False, genero_c=[])
bk = metodo_planif_fam_adulto_eess(df_data, actividad ='11.Abstinencia Periodica Billings', td_n=['D'], dx_n=['Z30093'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30493'], lab_c=[], lab_isnull_c=False, genero_c=[])
bl = metodo_planif_fam_adulto_eess(df_data, actividad ='12.Abstinencia Periodica Ritmo', td_n=['D'], dx_n=['Z30092'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30492'], lab_c=[], lab_isnull_c=False, genero_c=[])
bm = metodo_planif_fam_adulto_eess(df_data, actividad ='13.Abstinencia Periodica Días Fijos', td_n=['D'], dx_n=['Z30094'], lab_n=[], lab_isnull_n=False, genero_n=[], td_c=['D'], dx_c=['Z30494'], lab_c=[], lab_isnull_c=False, genero_c=[])
bn = metodo_planif_fam_adulto_eess(df_data, actividad ='Anticoncepción Oral de Emergencia/YUZPE', td_n=['D'], dx_n=['Z3091'], lab_n=[], lab_isnull_n=False, genero_n=['F'], td_c=['D'], dx_c=['Z3091'], lab_c=[], lab_isnull_c=False, genero_c=['F'])
bo = metodo_planif_fam_adulto_eess(df_data, actividad ='Anticoncepción Oral de Emergencia/Progestageno', td_n=['D'], dx_n=['Z3092'], lab_n=[], lab_isnull_n=False, genero_n=['F'], td_c=['D'], dx_c=['Z3092'], lab_c=[], lab_isnull_c=False, genero_c=['F'])

In [ ]:
list_planFam = [ba,bb,bc,bd,be,bf,bg,bh,bi,bj,bk,bl,bm,bn,bo]

In [ ]:
# concatenamos
df_list_planFam = pd.concat(list_planFam, ignore_index=True)
df_list_planFam

In [ ]:
# union de diagnosticos, pacientes y eess
df_reporte_adulto_planFam_eess = pd.merge(df_list_planFam,df_eess, left_on='Id_Establecimiento', right_on='Id_eess', how='inner')
df_reporte_adulto_planFam_eess

In [ ]:
df_reporte_adulto_planFam_eess = df_reporte_adulto_planFam_eess[['Sector','Dpto','Provincia','Distrito','Desc_UE','Red', 'MicroRed','Nombre_eess', 'Cat','Actividad','Anio', 'Mes', 'Activ_Nuevas','Insumo_Nuevas', 'Activ_Continuadoras','Insumo_Continuadoras']]
df_reporte_adulto_planFam_eess

In [ ]:
# genera archivo csv general
df_reporte_adulto_planFam_eess.to_csv('reporte_adulto_planFam_eess_16122022.csv', index=False, encoding='UTF-8')

In [ ]:
# eliminamos variables 
del ba,bb,bc,bd,be,bf,bg,bh,bi,bj,bk,bl,bm,bn,bo
del df_list_planFam
del df_reporte_adulto_planFam_eess

# Sesiones

In [ ]:
# Funcion para calcular el total por genero y por grupo etareo de las actividades de etapa de vida adulto mayor
def curso_vida_adulto_sesiones(df,**kwargs):
    # Parametros
    act = kwargs['actividad']
    dx = kwargs['dx']
    dx2 = kwargs['dx2']
    correlativo_lab = kwargs['correlativo_lab']

    df_GE = df[df['Anio_Actual_Paciente'].between(30,59)]
    if dx:
        df_PAMS1 = df_GE[(df_GE.Codigo_Item.isin(dx)) & (df_GE.Id_Correlativo_Lab.isin(correlativo_lab))]
    else:
        print('ingrese dx')
    
    if dx2:
        df_PAMS2 = df_GE[(df_GE.Codigo_Item.isin(dx2)) & (df_GE.Id_Correlativo_Lab.isin(correlativo_lab))]
    else:
        print('ingrese dx2')
    
    df_PAMS = pd.merge(df_PAMS1,df_PAMS2, on=['Id_Cita'],how='inner')
    
    if not (df_PAMS.empty): #activos nuevos
        df_PAMS['Valor_Lab_x'] = df_PAMS['Valor_Lab_x'].str.extract('(\d+(?:\.\d+)?)')
        df_PAMS['Valor_Lab_x'] = df_PAMS['Valor_Lab_x'].fillna(0)
        df_PAMS = df_PAMS.astype({'Valor_Lab_x':'float64'})
        #df_PAMS = df_PAMS.reset_index()
        # agrupamos y contamos por genero
        df_PAMS = df_PAMS.groupby(['Anio_x','Mes_x'])['Valor_Lab_x'].agg(['count','sum']).reset_index()
        df_PAMS = df_PAMS.rename({'Anio_x':'Anio','Mes_x':'Mes','count':'Nro','sum':'Beneficiarios'}, axis=1)

    else:
        df_PAMS = pd.DataFrame(columns=['Anio','Mes','Nro','Beneficiarios'], index=range(1))
          

    df_PAMS = df_PAMS[~df_PAMS.Mes.isnull()]
    # df_PAMS = df_PAMS.astype(int)
    # df_PAMS =  df_PAMS.sort_values(['Anio','Mes'])
    df_PAMS = df_PAMS.assign(Actividad=act)
    # df_PAMS = df_PAMS[df_PAMS.Mes!=0]

    return df_PAMS

In [ ]:
p = curso_vida_adulto_sesiones(df_data, actividad = '01.Sesiones Demostrativas', dx=['C0010'], dx2=['U0041'], correlativo_lab=[1])
q = curso_vida_adulto_sesiones(df_data, actividad = '02.Sesiones Educativas', dx=['C0009'], dx2=['U0041'], correlativo_lab=[1])
r = curso_vida_adulto_sesiones(df_data, actividad = '03.Campañas', dx=['U114'], dx2=['U0041'], correlativo_lab=[1])
s = curso_vida_adulto_sesiones(df_data, actividad = '04.Talleres de Habilidades Sociales', dx=['U122'], dx2=['U0041'], correlativo_lab=[1])

In [ ]:
list_sesiones =[p,q,r,s]

In [ ]:
# concatenamos
df_list_sesiones = pd.concat(list_sesiones, ignore_index=True)
df_list_sesiones

In [ ]:
# genera archivo csv general
df_list_sesiones.to_csv('reporte_adulto_sesiones_16122022.csv', index=False, encoding='UTF-8')

# OTRAS CONSEJERÍA 

In [ ]:
# Funcion para calcular el total por genero y por grupo etareo de las actividades de etapa de vida adulto mayor
def curso_vida_adulto_consejerias(df,**kwargs):
    # Parametros
    act = kwargs['actividad']
    dx = kwargs['dx']

    df_GE = df[df['Anio_Actual_Paciente'].between(30,59)]
    if dx:
        df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx))]
        df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita']) 
    else:
        print('ingrese dx')
    
    if not (df_PAMS.empty): #activos nuevos
        # agrupamos y contamos por genero
        df_PAMS = df_PAMS.groupby(['Anio','Mes'])['Id_Cita'].agg(['count']).reset_index()
        df_PAMS = df_PAMS.rename({'count':'Total'}, axis=1)

    else:
        df_PAMS = pd.DataFrame(columns=['Anio','Mes','Total'], index=range(1))
          
    df_PAMS = df_PAMS[~df_PAMS.Mes.isnull()]
    # df_PAMS = df_PAMS.astype(int)
    # df_PAMS =  df_PAMS.sort_values(['Anio','Mes'])
    df_PAMS = df_PAMS.assign(Actividad=act)
    # df_PAMS = df_PAMS[df_PAMS.Mes!=0]

    return df_PAMS

In [ ]:
# Funcion para calcular el total por genero y por grupo etareo de las actividades de etapa de vida adulto mayor
def curso_vida_adulto_consejerias_eess(df,**kwargs):
    # Parametros
    act = kwargs['actividad']
    dx = kwargs['dx']

    df_GE = df[df['Anio_Actual_Paciente'].between(30,59)]
    if dx:
        df_PAMS = df_GE[(df_GE.Codigo_Item.isin(dx))]
        df_PAMS = df_PAMS.drop_duplicates(subset=['Id_Cita']) 
    else:
        print('ingrese dx')
    
    if not (df_PAMS.empty): #activos nuevos
        # agrupamos y contamos por genero
        df_PAMS = df_PAMS.groupby(['Anio','Mes','Id_Establecimiento'])['Id_Cita'].agg(['count']).reset_index()
        df_PAMS = df_PAMS.rename({'count':'Total'}, axis=1)

    else:
        df_PAMS = pd.DataFrame(columns=['Anio','Mes','Id_Establecimiento','Total'], index=range(1))
          
    df_PAMS = df_PAMS[~df_PAMS.Mes.isnull()]
    # df_PAMS = df_PAMS.astype(int)
    # df_PAMS =  df_PAMS.sort_values(['Anio','Mes'])
    df_PAMS = df_PAMS.assign(Actividad=act)
    # df_PAMS = df_PAMS[df_PAMS.Mes!=0]

    return df_PAMS

In [ ]:
aa = curso_vida_adulto_consejerias(df_data, actividad = '01.Consejería Nutricional', dx=['99403.01'])
ab = curso_vida_adulto_consejerias(df_data, actividad = '02.Consejería Preventiva en Factores de Riesgo para el Cáncer', dx=['99402.08'])
ac = curso_vida_adulto_consejerias(df_data, actividad = '03.Consejería en Prevención de Enfermedades no Transmisibles', dx=['99401.31'])
ad = curso_vida_adulto_consejerias(df_data, actividad = '04.Consejería en Habilidades Sociales', dx=['99401.15'])
ae = curso_vida_adulto_consejerias(df_data, actividad = '05.Examen de Agudeza Visual Cuantitativa Bilateral', dx=['99173'])
af = curso_vida_adulto_consejerias(df_data, actividad = '06.Tamizaje Ocular con Instrumento Bilateral', dx=['99174'])

In [ ]:
list_consejeria =[aa,ab,ac,ad,ae,af]

In [ ]:
# concatenamos
df_list_consejeria = pd.concat(list_consejeria, ignore_index=True)
df_list_consejeria

In [ ]:
# genera archivo csv general
df_list_consejeria.to_csv(ruta_datos+'/Hisminsa_2022/consolidados/adulto/reporte_adulto_consejeria_16122022.csv', index=False, encoding='UTF-8')

# consejeria por eess

In [ ]:
ca = curso_vida_adulto_consejerias_eess(df_data, actividad = '01.Consejería Nutricional', dx=['99403.01'])
cb = curso_vida_adulto_consejerias_eess(df_data, actividad = '02.Consejería Preventiva en Factores de Riesgo para el Cáncer', dx=['99402.08'])
cc = curso_vida_adulto_consejerias_eess(df_data, actividad = '03.Consejería en Prevención de Enfermedades no Transmisibles', dx=['99401.31'])
cd = curso_vida_adulto_consejerias_eess(df_data, actividad = '04.Consejería en Habilidades Sociales', dx=['99401.15'])
ce = curso_vida_adulto_consejerias_eess(df_data, actividad = '05.Examen de Agudeza Visual Cuantitativa Bilateral', dx=['99173'])
cf = curso_vida_adulto_consejerias_eess(df_data, actividad = '06.Tamizaje Ocular con Instrumento Bilateral', dx=['99174'])

In [ ]:
list_consejeria_eess =[ca,cb,cc,cd,ce,cf]

In [ ]:
# concatenamos
df_list_consejeria_eess = pd.concat(list_consejeria_eess, ignore_index=True)
df_list_consejeria_eess

In [ ]:
# union de diagnosticos, pacientes y eess
df_reporte_adulto_consejeria_eess = pd.merge(df_list_consejeria_eess,df_eess, left_on='Id_Establecimiento', right_on='Id_eess', how='inner')
df_reporte_adulto_consejeria_eess

In [ ]:
# genera archivo csv general
df_reporte_adulto_consejeria_eess.to_csv('reporte_adulto_consejeria_eess_16122022.csv', index=False, encoding='UTF-8')